In [ ]:
import os
import re
import functools
import numpy as np
import pandas as pd
import pickle5 as pickle

In [ ]:
# CURR_PATH = os.getcwd()
CURR_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/LSTM'
DATA_PATH = '/home/svetlanamaslenkova/Documents/data/physionet.org/files/mimiciv/2.0/'
DF_PATH = CURR_PATH +'/dataframes_2/'
PKL_PATH = CURR_PATH+'/pickles_2/'
DF_PATH = CURR_PATH+'/dataframes_2/'
ICU_PATH = DATA_PATH + 'icu/'
HOSP_PATH = DATA_PATH + 'hosp/'

# Loading data

## Loading MIMIC IV tables

### hosp

patients

In [ ]:
import pandas as pd

data_patients = pd.read_csv(HOSP_PATH+'patients.csv')

data_patients.columns = data_patients.columns.str.lower()

data_patients.head(5)

admissions

In [ ]:
import pandas as pd

data_admissions = pd.read_csv(HOSP_PATH+'admissions.csv')

data_admissions.columns = data_admissions.columns.str.lower()

data_admissions.head(2)

In [ ]:
data_admissions['hadm_id'] = data_admissions['hadm_id'].astype(int)
# change charttime column to datettime type
format_ = '%Y-%m-%d %H:%M:%S'
# data_vitals['charttime'] = pd.to_datetime(data_vitals['charttime'], format=format_)
data_admissions['admittime'] = pd.to_datetime(data_admissions['admittime'], format=format_)
data_admissions['dischtime'] = pd.to_datetime(data_admissions['dischtime'], format=format_)

In [ ]:
data_admissions.info()

omr

In [ ]:
import pandas as pd

data_omr = pd.read_csv(HOSP_PATH+'omr.csv')
data_omr.columns = data_omr.columns.str.lower()
format_ = '%Y-%m-%d %H:%M:%S'
data_omr['chartdate'] = pd.to_datetime(data_omr['chartdate'], format=format_)
print(data_omr.shape)
data_omr.head(5)

In [ ]:
data_omr.result_name.unique()

weight_data

In [ ]:
weight_data = data_omr[data_omr.result_name.isin(['Weight', 'Weight (Lbs)'])]
weight_data['result_value'] = weight_data['result_value'].astype(float)

weight_data['weight_kg'] = [np.round(v * 0.45359237) for v in weight_data.loc[:, 'result_value']]
weight_data = data_icustays[['subject_id']].merge(weight_data[['subject_id', 'chartdate', 'weight_kg']]).drop_duplicates()

In [ ]:
weight_data.describe()

In [ ]:
# remove outliers (incorectly documented weight) 
weight_data = weight_data[~(weight_data.weight_kg>300)]

In [ ]:
# get the most recent to icu stay weight of patients 
weight_data_icu = data_icustays[['subject_id', 'hadm_id',	'stay_id', 'intime', 'outtime']]\
                        .merge(weight_data, on='subject_id')

weight_data_icu_prev = weight_data_icu[(weight_data_icu.chartdate > weight_data_icu.intime - pd.Timedelta(360, 'd'))&(weight_data_icu.chartdate < weight_data_icu.outtime)]\
                                    .sort_values(['subject_id', 'intime', 'chartdate'], ascending=False)\
                                        .drop_duplicates('stay_id', keep='first')
print(weight_data_icu_prev.shape)

In [ ]:
# get weight up to 180 days after outtime 
weight_data_icu_future = weight_data_icu[(weight_data_icu.chartdate >= weight_data_icu.intime)&(weight_data_icu.chartdate < weight_data_icu.outtime + pd.Timedelta(180, 'd'))]\
                                .sort_values(['subject_id', 'intime', 'chartdate'], ascending=False)\
                                    .drop_duplicates('stay_id', keep='last')
print(weight_data_icu_future.shape)

In [ ]:
# merge to dataframes and fillna in weight_data_icu_prev with values from future 
weight_data_icu = weight_data_icu_prev.merge(weight_data_icu_future.rename(columns={'stay_id':'stay_id_f', 'chartdate':'chartdate_f', 'weight_kg':'weight_kg_f'}).drop(columns=['intime', 'outtime']), how='outer')
weight_data_icu['weight_kg'] = weight_data_icu['weight_kg'].fillna(weight_data_icu['weight_kg_f'])
weight_data_icu['chartdate'] = weight_data_icu['chartdate'].fillna(weight_data_icu['chartdate_f'])
weight_data_icu['stay_id'] = weight_data_icu['stay_id'].fillna(weight_data_icu['stay_id_f'])
weight_data_icu


In [ ]:
print(weight_data_icu.hadm_id.unique().shape[0])
weight_data_icu.head()

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'weight_data_icu.pkl', 'wb') as f:
    pickle.dump(weight_data_icu, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'weight_data_icu.pkl', 'rb') as f:
    weight_data_icu = pickle.load(f)

d_icd_diagnoses

In [ ]:
import pandas as pd

data_d_icd_diagnoses = pd.read_csv(HOSP_PATH+'d_icd_diagnoses.csv')

data_d_icd_diagnoses.columns = data_d_icd_diagnoses.columns.str.lower()

data_d_icd_diagnoses.head()

diagnoses_icd

In [ ]:
import pandas as pd

data_diagnoses = pd.read_csv(HOSP_PATH+'diagnoses_icd.csv')

data_diagnoses.columns = data_diagnoses.columns.str.lower()

data_diagnoses.head()

labitems

In [ ]:
import pandas as pd

data_labitems = pd.read_csv(HOSP_PATH+'d_labitems.csv')

data_labitems.columns = data_labitems.columns.str.lower()
data_labitems.label = data_labitems.label.str.lower()
labitems = data_labitems.copy()
data_labitems.head(2)

labevents

In [ ]:
# take item ids of all needed features
creatinine_id = labitems[labitems.label.str.contains('creatinine').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
creatinine_serum_id = labitems[labitems.label.str.contains('creatinine, serum').fillna(False)].itemid.to_list()
urine_volume_id = labitems[labitems.label.str.contains('urine volume').fillna(False)].itemid.to_list()
anion_gap_id = labitems[labitems.label.str.contains('anion').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
albumin_id = labitems[labitems.label.str.contains('albumin').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
bands_id = labitems[labitems.label.str.contains('bands').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
bilirubin_id = labitems[labitems.label.str.contains('bilirubin').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
hematocrit_id = labitems[labitems.label.str.contains('hematocrit').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
lactate_id = labitems[labitems.label.str.contains('lactate').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
sodium_id = labitems[labitems.label.str.contains('sodium').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
bicarbonate_id = labitems[labitems.label.str.contains('bicarbonate').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
bun_id = labitems[labitems.label.str.contains('nitrogen').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
calcium_id = labitems[labitems.label.str.contains('calcium').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
chloride_id = labitems[labitems.label.str.contains('chloride').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
hemoglobin_id = list(set(labitems[labitems.label.str.contains('hemoglobin').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()) - set(['51212']))
inr_id = labitems[labitems.label.str.contains('inr').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
platelet_id = labitems[labitems.label.str.contains('platelet').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
potassium_id = labitems[labitems.label.str.contains('potassium').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
pt_id = ['51274', '52921', '52163', '52164']
ppt_id = ['51275', '52923', '52165', '52166', '52166', '52167']
wbc_id = labitems[labitems.label.str.contains('wbc').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()
glucose_id = labitems[labitems.label.str.contains('glucose').fillna(False) & labitems.fluid.isin(['Blood', 'Urine'])].itemid.to_list()

In [ ]:
lab_itemids = [*creatinine_id, *creatinine_serum_id, *urine_volume_id, *anion_gap_id, *albumin_id, *bands_id, *bilirubin_id, *hematocrit_id, *lactate_id,\
    *sodium_id, *bicarbonate_id, *bun_id, *calcium_id, *chloride_id, *hemoglobin_id, *inr_id, *platelet_id, *potassium_id, *pt_id, *ppt_id, *wbc_id, *glucose_id]
# all needed itemids    
len(set(lab_itemids))

Load labevents table by chunks (since it is very heavy):

In [ ]:
%%time
my_chunk = 1000000

# create the iterator
iter_csv = pd.read_csv(HOSP_PATH+'labevents.csv', 
        header=0, usecols=['subject_id', 'hadm_id', 'specimen_id', 'itemid','flag', 'charttime', 'valuenum', 'valueuom'],
                            dtype={'subject_id':'uint32', 'flag':'str', 'valueuom':'category'},
                       engine='c', keep_default_na=False,  skip_blank_lines=True, low_memory=False, iterator=True, chunksize=my_chunk)

# concatenate according to a filter to our result dataframe
df_result_lab = pd.concat(
    [chunk[chunk.itemid.isin(lab_itemids)] 
    for chunk in iter_csv])

In [ ]:
labevents = df_result_lab.copy()
labevents = labevents.merge(data_labitems[['itemid', 'label', 'fluid', 'category']])

In [ ]:
labevents['hadm_id'] = pd.to_numeric(labevents['hadm_id'], errors='coerce')
labevents['hadm_id'] = labevents['hadm_id'].fillna(0).astype(int)

# change charttime column to datettime type
format_ = '%Y-%m-%d %H:%M:%S'
labevents['charttime'] = pd.to_datetime(labevents['charttime'], format=format_)

# removing duplicated rows and rows with no valuenum
labevents = labevents.drop_duplicates()
labevents = labevents[~labevents.valuenum.isna()]

labevents = labevents[~labevents['valuenum'].str.contains("[a-zA-Z]+$")]
labevents['valuenum'] = pd.to_numeric(labevents['valuenum'], errors='coerce')
labevents = labevents.dropna(subset=['valuenum'])

labevents.flag = labevents.flag.astype(str)

labevents.shape

In [ ]:
labevents = labevents.merge(data_admissions[['subject_id',	'hadm_id', 'admittime', 'dischtime']])

In [ ]:
labevents.info()

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'labevents.pkl', 'wb') as f:
    pickle.dump(labevents, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'labevents.pkl', 'rb') as f:
    labevents = pickle.load(f)

labevents table with createnine test results only:

In [ ]:
creatinine_blood_id = [50912, 52546]
labevents_creatinine = labevents[labevents.itemid.isin(creatinine_blood_id)]

In [ ]:
labevents_creatinine['day_id'] = labevents_creatinine['charttime'] - labevents_creatinine['admittime']
labevents_creatinine['day_id'] = [d.days for d in labevents_creatinine.loc[:, 'day_id']]

In [ ]:
labevents_creatinine['_12h_window_id'] = labevents_creatinine['charttime'] - labevents_creatinine['admittime']
labevents_creatinine['_12h_window_id'] = [(d.days*24 + d.seconds//3600)//12 for d in labevents_creatinine.loc[:, '_12h_window_id']]

In [ ]:
labevents_creatinine['hour_id'] = labevents_creatinine['charttime'] - labevents_creatinine['admittime']
labevents_creatinine['hour_id'] = [(d.days*24 + d.seconds//3600) for d in labevents_creatinine.loc[:, 'hour_id']]

In [ ]:
labevents_creatinine.sort_values(['hadm_id', 'charttime']).head()

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'labevents_creatinine.pkl', 'wb') as f:
    pickle.dump(labevents_creatinine, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'labevents_creatinine.pkl', 'rb') as f:
    labevents_creatinine = pickle.load(f)

prescriptions

In [ ]:
import pandas as pd

data_prescriptions = pd.read_csv(HOSP_PATH+'prescriptions.csv')

data_prescriptions.columns = data_prescriptions.columns.str.lower()

# data_prescriptions['ndc'] = data_prescriptions['ndc'].astype('uint64')

data_prescriptions.head(5)

### ICU

d_items

In [ ]:
import pandas as pd

data_d_items = pd.read_csv(ICU_PATH + 'd_items.csv')

data_d_items.columns = data_d_items.columns.str.lower()

data_d_items.head()

Load chartevents table in chunks (since it is very heavy):

In [ ]:
%%time
import pandas as pd

my_chunk = 1000000

# create the iterator
iter_csv = pd.read_csv(ICU_PATH + 'chartevents.csv', header=0, usecols=['subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value', 'valuenum', 'valueuom'], \
                        dtype={'subject_id':'uint32', 'hadm_id':'uint32', 'stay_id':'uint32', 'valueuom':'category'},\
                            engine='c', keep_default_na=False, skip_blank_lines=True, low_memory=False, iterator=True, chunksize=my_chunk )              

# concatenate according to a filter to our result dataframe
df_result = pd.concat(\
    [chunk for chunk in iter_csv])
         

In [ ]:
# data_chartevents = df_result.copy()
# data_chartevents.columns = data_chartevents.columns.str.lower()
# format_ = '%Y-%m-%d %H:%M:%S'
# data_chartevents['charttime'] = pd.to_datetime(data_chartevents['charttime'], format=format_)
data_chartevents['valuenum'] = data_chartevents['valuenum'].replace(r'[a-zA-Z]', '').astype(float)

In [ ]:
%%time
# data_chartevents = data_chartevents.merge(data_d_items[['itemid', 'label', 'abbreviation', 'category', 'unitname', 'param_type']])
print('unique stays: ', data_chartevents.stay_id.unique().shape)
data_chartevents.head()

In [ ]:
data_chartevents.category.unique()

In [ ]:
unique_chart_items = data_chartevents.drop_duplicates('itemid', keep='first')

In [ ]:
# print items
print('itemid | label | category | valuenum')
for _, row in  unique_chart_items.sort_values('category').iterrows():
    print(row.itemid, '|', row.label, '|', row.category, '|', row.value)

In [ ]:
for _, row in data_d_items[data_d_items.category=='Routine Vital Signs'].iterrows():
    print(row.itemid, '|', row.label, )

Lab results:

In [ ]:
labs_itemids = data_d_items[data_d_items.category=='Labs'].itemid.unique()
labs_itemids

In [ ]:
chartevents_labs = data_chartevents[data_chartevents.itemid.isin(labs_itemids)]
chartevents_labs = chartevents_labs.merge(data_d_items[['itemid', 'label', 'abbreviation', 'category', 'unitname', 'param_type']])
chartevents_labs['valuenum'] = chartevents_labs['valuenum'].astype(float)

In [ ]:
chartevents_labs.info()

In [ ]:
chartevents_labs.head()

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'chartevents_labs.pkl', 'wb') as f:
    pickle.dump(chartevents_labs, f, protocol=pickle.HIGHEST_PROTOCOL)

Creatinine

In [ ]:
for _, row in data_d_items[data_d_items.category=='Labs'].iterrows():
    print(row.itemid, '|', row.label, )

In [ ]:
itemid_scr = 220615
chartevents_creatinine = chartevents_labs[chartevents_labs.itemid==itemid_scr]

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'chartevents_creatinine.pkl', 'wb') as f:
    pickle.dump(chartevents_creatinine, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'chartevents_creatinine.pkl', 'rb') as f:
    chartevents_creatinine = pickle.load(f)

data_datetimeevents

In [ ]:
import pandas as pd

data_datetimeevents = pd.read_csv(ICU_PATH + 'datetimeevents.csv')

data_datetimeevents.columns = data_datetimeevents.columns.str.lower()

data_datetimeevents.head()

icustays

In [ ]:
import pandas as pd

data_icustays = pd.read_csv(ICU_PATH + 'icustays.csv')

data_icustays.columns = data_icustays.columns.str.lower()

data_icustays.head()

In [ ]:
data_icustays['hadm_id'] = data_icustays['hadm_id'].astype(int)
# change charttime column to datettime type
format_ = '%Y-%m-%d %H:%M:%S'
data_icustays['intime'] = pd.to_datetime(data_icustays['intime'], format=format_)
data_icustays['outtime'] = pd.to_datetime(data_icustays['outtime'], format=format_)

In [ ]:
print('n of unique patients: ', (data_icustays.subject_id.unique().shape[0]))
print('n of unique admissions: ', (data_icustays.hadm_id.unique().shape[0]))
print('n of unique ICU stays: : ', (data_icustays.stay_id.unique().shape[0]))

In [ ]:
S = data_icustays.merge(data_admissions[['subject_id', 'hadm_id', 'admittime']], how='left')
S['day_id'] = S['intime'] - S['admittime']
S['day_id'] = [d.days for d in S.loc[:, 'day_id']]
S

In [ ]:
# filter out icu stays with less than 2 days 
data_icustays = data_icustays[data_icustays.los>=2].sort_values(['subject_id', 'hadm_id', 'stay_id'])
print(data_icustays.shape)
data_icustays.head(1)

inputevents

In [ ]:
%%time
import pandas as pd

data_inputevents = pd.read_csv(ICU_PATH + 'inputevents.csv')

data_inputevents.columns = data_inputevents.columns.str.lower()

data_inputevents.head().iloc[:, :14]

In [ ]:
data_inputevents = data_inputevents.merge(data_icustays[['stay_id', 'intime']], on='stay_id')

In [ ]:
print('itemid','|', 'label','|', 'abbreviation','|', 'category')
for _,row in data_d_items[data_d_items.linksto=='inputevents'].iterrows():
    print(row.itemid,'|', row.label,'|', row.abbreviation,'|', row.category)

In [ ]:
data_inputevents = data_inputevents.merge(data_d_items[['itemid', 'label', 'abbreviation', 'category']], on='itemid')

In [ ]:
# change charttime column to datettime type
format_ = '%Y-%m-%d %H:%M:%S'
data_inputevents['starttime'] = pd.to_datetime(data_inputevents['starttime'], format=format_)
data_inputevents['endtime'] = pd.to_datetime(data_inputevents['endtime'], format=format_)

Ingridients

In [ ]:
import pandas as pd

data_ingredientevents = pd.read_csv(ICU_PATH + 'ingredientevents.csv')

data_ingredientevents.columns = data_ingredientevents.columns.str.lower()

data_ingredientevents.head()

In [ ]:
data_ingredientevents.merge(data_d_items[['itemid', 'label',	'abbreviation']])

outputevents

Load outputevents in chuncks:

In [ ]:
%%time
import pandas as pd

data_outputevents = pd.read_csv(ICU_PATH + 'outputevents.csv')

data_outputevents = data_outputevents.merge(data_d_items, on='itemid')

data_outputevents.columns = data_outputevents.columns.str.lower()
data_outputevents['hadm_id'] = data_outputevents['hadm_id'].astype(int)
# change charttime column to datettime type
format_ = '%Y-%m-%d %H:%M:%S'
data_outputevents['charttime'] = pd.to_datetime(data_outputevents['charttime'], format=format_)

data_outputevents.head()

In [ ]:
print('itemid | label | abbreviation | category')
for _, row in data_outputevents.drop_duplicates('itemid').iterrows():
    print(f'{row.itemid} | {row.label} |  {row.abbreviation} | {row.category}')

In [ ]:
data_outputevents = data_outputevents.merge(data_icustays[['subject_id',	'hadm_id', 'stay_id', 'intime']])\
                                        .merge(data_admissions[['subject_id',	'hadm_id', 'admittime']])\
                                            .sort_values(['subject_id','hadm_id'])

In [ ]:
# merge labevents with icu_stays to get icu stay column
O = data_outputevents.merge(data_icustays[['subject_id', 'hadm_id',	'stay_id', 'intime', 'outtime']])\
                        .merge(data_admissions[['subject_id',	'hadm_id', 'admittime']])\
                            .sort_values(['subject_id', 'hadm_id', 'charttime'])

# # remove wrong rows with inconsistency between intime, outtime and charttime
# O = O[(O.charttime > O.intime)&(O.charttime < O.outtime)]
# # get day ids for starting from admittime and intime
# O['day_id'] = O['charttime'] - O['admittime']
# O['day_id'] = [d.days for d in O.loc[:, 'day_id']]
# O['icu_day_id'] = O['charttime'] - O['intime']
# O['icu_day_id'] = [d.days for d in O.loc[:, 'icu_day_id']]

In [ ]:
data_outputevents['day_id'] = data_outputevents['charttime'] - data_outputevents['admittime']
data_outputevents['day_id'] = [d.days for d in data_outputevents.loc[:, 'day_id']]

In [ ]:
data_outputevents['_12h_window_id'] = data_outputevents['charttime'] - data_outputevents['admittime']
data_outputevents['_12h_window_id'] = [(d.days*24 + d.seconds//3600)//12 for d in data_outputevents.loc[:, '_12h_window_id']]

data_outputevents['hour_id'] = data_outputevents['charttime'] - data_outputevents['admittime']
data_outputevents['hour_id'] = [(d.days*24 + d.seconds//3600) for d in data_outputevents.loc[:, 'hour_id']]

In [ ]:
%%time
data_outputevents['icu_day_id'] = data_outputevents['charttime'] - data_outputevents['intime']
data_outputevents['icu_day_id'] = [d.days for d in data_outputevents.loc[:, 'icu_day_id']]

data_outputevents['icu_12h_window_id'] = data_outputevents['charttime'] - data_outputevents['intime']
data_outputevents['icu_12h_window_id'] = [(d.days*24 + d.seconds//3600)//12 for d in data_outputevents.loc[:, 'icu_12h_window_id']]

data_outputevents['icu_hour_id'] = data_outputevents['charttime'] - data_outputevents['intime']
data_outputevents['icu_hour_id'] = [(d.days*24 + d.seconds//3600) for d in data_outputevents.loc[:, 'icu_hour_id']]

In [ ]:
print(data_outputevents.shape)
data_outputevents.head(5)

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'data_outputevents.pkl', 'wb') as f:
    pickle.dump(data_outputevents, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'data_outputevents.pkl', 'rb') as f:
    data_outputevents = pickle.load(f)

procedureevents

In [ ]:
import pandas as pd

data_procedureevents = pd.read_csv(ICU_PATH + 'procedureevents.csv')

data_procedureevents.columns = data_procedureevents.columns.str.lower()

data_procedureevents.head()

## NDC map

Load the table with mapping NDC codes to ATC codes:

In [ ]:
import pandas as pd

ndc_map = pd.read_csv('/home/svetlanamaslenkova/Documents/data/FDA NDC directory with atc5 atc4 ingredients (2020_06_17)/ndc_map 2020_06_17 (atc5 atc4 ingredients).csv')

ndc_map.columns = ndc_map.columns.str.lower()

ndc_map.atc4_name = ndc_map.atc4_name.astype('str')

ndc_map = ndc_map[~ndc_map.atc4.isna()]

print(ndc_map.shape)
ndc_map.head(6)

## Loading prepared dataframes

In [ ]:
with open('/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/pickles/' + 'pid_labevents.pkl', 'rb') as f:
    pid_labevents = pickle.load(f)
print('number of admissions from ICU: ', data_icustays[data_icustays.hadm_id.isin(pid_labevents.hadm_id.unique())].hadm_id.unique().shape[0])

In [ ]:
with open('/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/pickles/' + 'aki_status_df.pkl', 'rb') as f:
    final_aki_status = pickle.load(f)
final_aki_status = final_aki_status[~final_aki_status.isna()]
print('number of admissions from ICU: ', data_icustays[data_icustays.hadm_id.isin(final_aki_status.hadm_id.unique())].hadm_id.unique().shape[0])
final_aki_status.head()

In [ ]:
with open(PKL_PATH + 'pid_vitals.pkl', 'rb') as f:
    pid_vitals = pickle.load(f)
temp = pid_vitals[pid_vitals.vitals.str.contains(r'[0-9]')]
print('number of admissions from ICU with any information: ', data_icustays[data_icustays.hadm_id.isin(temp.hadm_id.unique())].hadm_id.unique().shape[0])
print('number of admissions from ICU: ', data_icustays[data_icustays.hadm_id.isin(pid_vitals.hadm_id.unique())].hadm_id.unique().shape[0])
pid_vitals.head()

In [ ]:
with open(PKL_PATH + 'pid_medications.pkl', 'rb') as f:
    pid_medications = pickle.load(f)
temp = pid_medications[pid_medications.medications.str.contains(r'[a-zA-Z]')]
print('number of admissions from ICU with any information: ', data_icustays[data_icustays.hadm_id.isin(temp.hadm_id.unique())].hadm_id.unique().shape[0])
print('number of admissions from ICU: ', data_icustays[data_icustays.hadm_id.isin(pid_medications.hadm_id.unique())].hadm_id.unique().shape[0])
pid_medications.head()

In [ ]:
with open(PKL_PATH + 'data_demographics.pkl', 'rb') as f:
    data_demographics = pickle.load(f)
temp = data_demographics[data_demographics.demographics.str.contains(r'M|F')]
print('number of admissions from ICU with any information: ', data_icustays[data_icustays.hadm_id.isin(temp.hadm_id.unique())].hadm_id.unique().shape[0])
print('number of admissions from ICU: ', data_icustays[data_icustays.hadm_id.isin(data_demographics.hadm_id.unique())].hadm_id.unique().shape[0])
data_demographics.head()

In [ ]:
with open('/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/pickles/' + 'hid_previous_diagnoses.pkl', 'rb') as f:
    df_previous_diags = pickle.load(f)
temp = df_previous_diags[df_previous_diags.previous_diags_icd.str.contains('D')]
print('number of admissions from ICU with any information: ', data_icustays[data_icustays.hadm_id.isin(temp.hadm_id.unique())].hadm_id.unique().shape[0])
print('number of admissions from ICU: ', data_icustays[data_icustays.hadm_id.isin(df_previous_diags.hadm_id.unique())].hadm_id.unique().shape[0])
df_previous_diags.head()

In [ ]:
with open(PKL_PATH + 'labevents_creatinine.pkl', 'rb') as f:
    labevents_creatinine = pickle.load(f)
print('number of admissions from ICU: ', data_icustays[data_icustays.hadm_id.isin(labevents_creatinine.hadm_id.unique())].hadm_id.unique().shape[0])
labevents_creatinine.head()

# Data analysis

In [ ]:
with open(PKL_PATH + 'aki_status_icu.pkl', 'rb') as f:
    aki_status_icu = pickle.load(f)
aki_status_icu.head()

In [ ]:
with open(PKL_PATH + 'aki_stage_labels.pkl', 'rb') as f:
    aki_stage_labels = pickle.load(f)
aki_stage_labels.head()   

In [ ]:
print('n of unique patients: ', (data_icustays.subject_id.unique().shape[0]))
print('n of unique admissions: ', (data_icustays.hadm_id.unique().shape[0]))
print('n of unique ICU stays: : ', (data_icustays.stay_id.unique().shape[0]))

The numbers are not consistent because one day a patient can have AKI 1 and the next day AKI 2, so the same patient will fall into two categories.

In [ ]:
# number of admissions with aki status available from ICU wards
# icu_aki_status = final_aki_status.merge(data_icustays, on='hadm_id', how='inner')
icu_aki_status = aki_stage_labels.copy()
print('number of admissions with aki status available from ICU wards: ', \
                    icu_aki_status.hadm_id.unique().shape[0])

# print('number of admissions in icu: ', data_icustays.hadm_id.unique().shape[0])
g_adm = aki_stage_labels.groupby('hadm_id').sum()
print('admissions with no AKI in ICU: ', g_adm[g_adm.ANY_AKI==0].index.shape[0])
print('admissions with AKI 1 in ICU: ', icu_aki_status[icu_aki_status.AKI_1==1].drop_duplicates(subset=['hadm_id']).shape[0])
print('admissions with AKI 2 in ICU: ', icu_aki_status[icu_aki_status.AKI_2==1].drop_duplicates(subset=['hadm_id']).shape[0])
print('admissions with AKI 3 in ICU: ', icu_aki_status[icu_aki_status.AKI_3==1].drop_duplicates(subset=['hadm_id']).shape[0])

In [ ]:
print('number of stays with aki status available from ICU wards: ', \
                    icu_aki_status.stay_id.unique().shape[0])
g_adm = aki_stage_labels.groupby('stay_id').sum()
print('admissions with no AKI in ICU: ', g_adm[g_adm.ANY_AKI==0].index.shape[0])
print('admissions with AKI 1 in ICU: ', icu_aki_status[icu_aki_status.AKI_1==1].drop_duplicates(subset=['stay_id']).shape[0])
print('admissions with AKI 2 in ICU: ', icu_aki_status[icu_aki_status.AKI_2==1].drop_duplicates(subset=['stay_id']).shape[0])
print('admissions with AKI 3 in ICU: ', icu_aki_status[icu_aki_status.AKI_3==1].drop_duplicates(subset=['stay_id']).shape[0])

Make labels for the second day in ICU:

In [ ]:
aki_stage_labels_second_day = aki_stage_labels[aki_stage_labels.icu_day_id==1]

In [ ]:
aki_stage_labels_second_day = aki_stage_labels_second_day.groupby('stay_id').sum().reset_index()[['subject_id', 'hadm_id', 'stay_id', 'AKI_1',	'AKI_2',	'AKI_3',	'ANY_AKI', 'NO_AKI']]

In [ ]:
aki_stage_labels_second_day['AKI_2'] = list(np.where((aki_stage_labels_second_day['AKI_3']==0), aki_stage_labels_second_day['AKI_2'], 0))
aki_stage_labels_second_day['AKI_1'] = list(np.where(((aki_stage_labels_second_day['AKI_3']==0)&(aki_stage_labels_second_day['AKI_2']==0)), aki_stage_labels_second_day['AKI_1'], 0))
aki_stage_labels_second_day['ANY_AKI'] = np.where(((aki_stage_labels_second_day['AKI_1']==0)&(aki_stage_labels_second_day['AKI_2']==0)&(aki_stage_labels_second_day['AKI_3']==0)), 0, 1)
aki_stage_labels_second_day['NO_AKI'] = np.where(((aki_stage_labels_second_day['AKI_1']==0)&(aki_stage_labels_second_day['AKI_2']==0)&(aki_stage_labels_second_day['AKI_3']==0)), 1, 0)

In [ ]:
aki_stage_labels_second_day['AKI_1'] = aki_stage_labels_second_day['AKI_1'].astype(bool).astype(int)
aki_stage_labels_second_day['AKI_2'] = aki_stage_labels_second_day['AKI_2'].astype(bool).astype(int)
aki_stage_labels_second_day['AKI_3'] = aki_stage_labels_second_day['AKI_3'].astype(bool).astype(int)
aki_stage_labels_second_day['ANY_AKI'] = aki_stage_labels_second_day['ANY_AKI'].astype(bool).astype(int)
aki_stage_labels_second_day['NO_AKI'] = aki_stage_labels_second_day['NO_AKI'].astype(bool).astype(int)

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'aki_stage_labels_second_day.pkl', 'wb') as f:
    pickle.dump(aki_stage_labels_second_day, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'aki_stage_labels_second_day.pkl', 'rb') as f:
    aki_stage_labels_second_day = pickle.load(f)

Here numbers are consistent because the labels are for the second day in ICU:

In [ ]:
print('number of stays with aki status available from ICU wards: ', \
                    aki_stage_labels_second_day.stay_id.unique().shape[0])
g_adm = aki_stage_labels_second_day.groupby('stay_id').sum()
print('admissions with no AKI in ICU: ', g_adm[g_adm.ANY_AKI==0].index.shape[0])
print('admissions with AKI 1 in ICU: ', aki_stage_labels_second_day[aki_stage_labels_second_day.AKI_1==1].stay_id.unique().shape[0])
print('admissions with AKI 2 in ICU: ', aki_stage_labels_second_day[aki_stage_labels_second_day.AKI_2==1].stay_id.unique().shape[0])
print('admissions with AKI 3 in ICU: ', aki_stage_labels_second_day[aki_stage_labels_second_day.AKI_3==1].stay_id.unique().shape[0])

In [ ]:
from matplotlib import pyplot as plt
icu_aki_status = aki_status_icu.copy()
g0 = icu_aki_status[(icu_aki_status.AKI_1==1)].sort_values(['hadm_id', 'day_id'])\
                        .drop_duplicates(['hadm_id'], keep='first')\
                            .groupby('day_id')\
                                .count()
index0 = g0.AKI_1.sort_values(ascending=False).index
values0 = g0.AKI_1.sort_values(ascending=False).values

g1 = icu_aki_status[(icu_aki_status.AKI_1==1)&(icu_aki_status.AKI_2==0)].sort_values(['hadm_id', 'day_id'])\
                        .drop_duplicates(['hadm_id'], keep='first')\
                            .groupby('day_id')\
                                .count()
index1 = g1.AKI_1.sort_values(ascending=False).index
values1 = g1.AKI_1.sort_values(ascending=False).values

g2 = icu_aki_status[(icu_aki_status.AKI_2==1)&(icu_aki_status.AKI_3==0)].sort_values(['hadm_id', 'day_id'])\
                        .drop_duplicates(['hadm_id'], keep='first')\
                            .groupby('day_id')\
                                .count()
index2 = g2.AKI_2.sort_values(ascending=False).index
values2 = g2.AKI_2.sort_values(ascending=False).values

g3 = icu_aki_status[icu_aki_status.AKI_3==1].sort_values(['hadm_id', 'day_id'])\
                        .drop_duplicates(['hadm_id'], keep='first')\
                            .groupby('day_id')\
                                .count()
index3 = g3.AKI_3.sort_values(ascending=False).index
values3 = g3.AKI_3.sort_values(ascending=False).values

fig, (ax2) = plt.subplots(1, 1, figsize=(12, 6), sharex=False) 

ax2.set_xticks(index1)
# ax2.set_yticks(np.arange(0, 3000, 100))
ax2.set_xlim([0,25])
ax2.set_xlabel('day_id')
ax2.set_ylabel('n of patients with AKI')
ax2.bar(index0, values0, width=0.9, label='AKI of any stage')
ax2.bar(index1, values1, width=0.9, label='AKI of stage 1')
ax2.bar(index2, values2, width=0.9, label='AKI stage 2')
ax2.bar(index3, values3, width=0.9, label='AKI stage 3')
ax2.set_title('Number of patients having the first AKI onset on a particular day of an admission')
ax2.grid(True)
ax2.legend()

plt.show()

In [ ]:
from turtle import color
from matplotlib import pyplot as plt
icu_aki_status = aki_status_icu.copy()

g0 = icu_aki_status[(icu_aki_status.AKI_1==1)].sort_values(['hadm_id', 'icu_12h_window_id'])\
                        .drop_duplicates(['stay_id'], keep='first')\
                            .groupby('icu_day_id')\
                                .count()
index0 = g0.AKI_1.sort_values(ascending=False).index
values0 = g0.AKI_1.sort_values(ascending=False).values

g1 = icu_aki_status[(icu_aki_status.AKI_1==1)&(icu_aki_status.AKI_2==0)].sort_values(['hadm_id', 'icu_12h_window_id'])\
                        .drop_duplicates(['stay_id'], keep='first')\
                            .groupby('icu_day_id')\
                                .count()
index1 = g1.AKI_1.sort_values(ascending=False).index
values1 = g1.AKI_1.sort_values(ascending=False).values

g2 = icu_aki_status[(icu_aki_status.AKI_2==1)&(icu_aki_status.AKI_3==0)].sort_values(['hadm_id', 'icu_12h_window_id'])\
                        .drop_duplicates(['stay_id'], keep='first')\
                            .groupby('day_id')\
                                .count()
index2 = g2.AKI_2.sort_values(ascending=False).index
values2 = g2.AKI_2.sort_values(ascending=False).values

g3 = icu_aki_status[icu_aki_status.AKI_3==1].sort_values(['hadm_id', 'icu_12h_window_id'])\
                        .drop_duplicates(['stay_id'], keep='first')\
                            .groupby('day_id')\
                                .count()
index3 = g3.AKI_3.sort_values(ascending=False).index
values3 = g3.AKI_3.sort_values(ascending=False).values

fig, (ax2) = plt.subplots(1, 1, figsize=(12, 6), sharex=False) 

ax2.set_xticks(index1)
# ax2.set_yticks(np.arange(0, 3000, 100))
ax2.set_xlim([0,25])
ax2.set_xlabel('icu_day_id')
ax2.set_ylabel('n of patients with AKI')
ax2.bar(index0, values0, width=0.9, label='AKI of any stage')
ax2.bar(index1, values1, width=0.9, label='AKI of stage 1')
ax2.bar(index2, values2, width=0.9, label='AKI stage 2')
ax2.bar(index3, values3, width=0.9, label='AKI stage 3')
ax2.set_title('Number of patients having the first AKI onset on a particular day of an ICU stay')
ax2.grid(True)
ax2.legend()

plt.show()

In [ ]:
# S = data_icustays.merge(data_admissions[['subject_id', 'hadm_id', 'admittime']], how='left')
# S['day_id'] = S['intime'] - S['admittime']
# S['day_id'] = [d.days for d in S.loc[:, 'day_id']]
S[S.day_id>0].stay_id.unique().shape[0]

In [ ]:
print(f'Number of patients who arrived to ICU on the first day of an admission:  {S[S.day_id==0].stay_id.unique().shape[0]}')
print(f'Number of patients who arrived to ICU after being admitted to the hospital:  {S[S.day_id>0].stay_id.unique().shape[0]}')

length of ICU stay

In [ ]:
data_icustays.head(1)

In [ ]:
print('n of unique ICU stays with length of stay at least than 2 days: : ', (data_icustays[data_icustays.los>=2].stay_id.unique().shape[0]))

In [ ]:
from matplotlib import pyplot as plt

print(np.round(data_icustays.los.min(),2) , np.round(data_icustays.los.max(), 2))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 4), sharex=False)
ax1.hist(data_icustays.los,  bins=50, range=(0,20),histtype ='bar', facecolor='#088F8F')
ax1.set_xticks(np.arange(0, 20))
ax1.set_xlabel('Number of days in ICU')
ax1.set_ylabel('Number of patients')
ax1.grid()

ax2.boxplot(data_icustays.los, vert=False)
ax2.set_xticks(np.arange(0, 50, 2 ))
ax2.set_xlim([0,50])
ax2.set_xlabel('Number of days in ICU')
ax2.grid()

fig.suptitle("Length of ICU stay", fontsize="x-large")

# AKI status for ICU patients

## Get urine output values

In [ ]:
with open(PKL_PATH + 'weight_data_icu.pkl', 'rb') as f:
    weight_data_icu = pickle.load(f)
weight_data_icu.head(1)

In [ ]:
print('itemid | label | value | valueuom | abbreviation | category')
for _, row in data_outputevents[data_outputevents.category=='Output'].drop_duplicates('itemid').iterrows():
    print(f'{row.itemid} | {row.label} | {row.value}  | {row.valueuom} | {row.abbreviation} | {row.category}')

In [ ]:
urine_output_ids = [226559, 226627, 226561]

In [ ]:
data_d_items[data_d_items.itemid.isin(urine_output_ids)]

In [ ]:
urine_output_df = data_outputevents[data_outputevents.itemid.isin(urine_output_ids)].sort_values(['subject_id',	'hadm_id',	'stay_id', 'charttime'])

urine_output_df['icu_6h_window_id'] = urine_output_df['charttime'] - urine_output_df['intime']
urine_output_df['icu_6h_window_id'] = [(d.days*24 + d.seconds//3600)//6 for d in urine_output_df.loc[:, 'icu_6h_window_id']]

In [ ]:
sum_urine_output_6h = urine_output_df[['subject_id', 'hadm_id',	'stay_id', 'icu_day_id',  'icu_6h_window_id', 'value']]\
                            .groupby(['subject_id', 'hadm_id', 'stay_id','icu_day_id', 'icu_6h_window_id'])\
                                .agg('sum').reset_index()\
                                    .merge(weight_data_icu[['subject_id', 'hadm_id', 'stay_id', 'weight_kg']]).sort_values(['subject_id','hadm_id','stay_id','icu_6h_window_id'])

sum_urine_output_12h = urine_output_df[['subject_id', 'hadm_id','stay_id', 'icu_day_id', 'icu_12h_window_id', 'value']]\
                            .groupby(['subject_id', 'hadm_id', 'stay_id','icu_day_id', 'icu_12h_window_id'])\
                                .agg('sum')\
                                    .reset_index().merge(weight_data_icu[['subject_id', 'hadm_id', 'stay_id', 'weight_kg']]).sort_values(['subject_id','hadm_id','stay_id','icu_12h_window_id'])
                                    
sum_urine_output_24h = urine_output_df[['subject_id', 'hadm_id','stay_id', 'icu_day_id', 'value']]\
                            .groupby(['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'])\
                                .agg('sum')\
                                    .reset_index().merge(weight_data_icu[['subject_id', 'hadm_id', 'stay_id', 'weight_kg']]).sort_values(['subject_id','hadm_id','stay_id','icu_day_id'])

In [ ]:
sum_urine_output_6h['urine_ml_kg_h_6h'] = sum_urine_output_6h['value'] / sum_urine_output_6h['weight_kg'] / 6
sum_urine_output_12h['urine_ml_kg_h_12h'] = sum_urine_output_12h['value'] / sum_urine_output_12h['weight_kg'] / 12
sum_urine_output_24h['urine_ml_kg_h_24h'] = sum_urine_output_24h['value'] / sum_urine_output_24h['weight_kg'] / 24

In [ ]:
sum_urine_output_6h['AKI_1'] = (sum_urine_output_6h.urine_ml_kg_h_6h < 0.5).astype(int).values
sum_urine_output_12h['AKI_2'] = (sum_urine_output_12h.urine_ml_kg_h_12h < 0.5).astype(int).values
sum_urine_output_24h['AKI_3'] = (sum_urine_output_24h.urine_ml_kg_h_24h < 0.3).astype(int).values

See the examples of urine output rates:

In [ ]:
print(sum_urine_output_6h.stay_id.unique().shape)
print(sum_urine_output_12h.stay_id.unique().shape)
print(sum_urine_output_24h.stay_id.unique().shape)

## Urine AKI critetia

In [ ]:
def get_aki_status_urine_out(stay_id, interval='24'):
    adm_urine_output_6h = sum_urine_output_6h[sum_urine_output_6h.stay_id==stay_id]
    adm_urine_output_12h = sum_urine_output_12h[sum_urine_output_12h.stay_id==stay_id]
    adm_urine_output_24h = sum_urine_output_24h[sum_urine_output_24h.stay_id==stay_id]
    AKI_1_list = []
    AKI_2_list = []
    AKI_3_list = []
    AKI_1_12h_list = []
    AKI_2_12h_list = []
    AKI_3_12h_list = []
    stay_id_list_1 = []
    stay_id_list_2 = []
    #days in icu
    try:
        if interval=='24':
            days = list(adm_urine_output_24h.icu_day_id.unique())
            arranged_days = list(np.arange(0, np.max(days) + 1))
        for day in arranged_days:
            if day in days:
                # i = arranged_days.index(day)
                if sum(adm_urine_output_6h[adm_urine_output_6h.icu_day_id==day].AKI_1.values)>0:
                    AKI_1_list.append(1)
                else:
                    AKI_1_list.append(0)
                if (sum(adm_urine_output_12h[adm_urine_output_12h.icu_day_id==day].AKI_2.values)>0)&(AKI_1_list[day]==True):
                    AKI_2_list.append(1)
                else:
                    AKI_2_list.append(0)
                if (sum(adm_urine_output_24h[adm_urine_output_24h.icu_day_id==day].AKI_3.values)>0)&(AKI_1_list[day]==True):
                    AKI_3_list.append(1)
                else:
                    AKI_3_list.append(0)
            else:
                AKI_1_list.append(0)
                AKI_2_list.append(0)
                AKI_3_list.append(0)
            stay_id_list_1.append(stay_id)
        dict_ = {'stay_id':stay_id_list_1, 'icu_day_id':arranged_days, 'AKI_1_urine':AKI_1_list, 'AKI_2_urine':AKI_2_list, 'AKI_3_urine':AKI_3_list}
        # not correct:
        if interval=='12':
            wind_6h = adm_urine_output_6h.icu_6h_window_id.unique()
            wind_12h = adm_urine_output_12h.icu_12h_window_id.unique()
            day_id_list = []
            for wind in np.arange(np.min(wind_12h), np.max(wind_12h)+1):
                if sum(adm_urine_output_6h[adm_urine_output_6h.icu_6h_window_id.isin(np.arange(wind*2, wind*2+2))].AKI_1.values)>0:
                    AKI_1_12h_list.append(1)
                else:
                    AKI_1_12h_list.append(0)
                if (sum(adm_urine_output_12h[adm_urine_output_12h.icu_12h_window_id==wind].AKI_2.values)>0)&(AKI_1_12h_list[wind]==True):
                    AKI_2_12h_list.append(1)
                else:
                    AKI_2_12h_list.append(0)
                if (sum(adm_urine_output_24h[adm_urine_output_24h.icu_day_id==wind//2].AKI_3.values)>0)&(AKI_1_list[day]==True):
                    AKI_3_12h_list.append(1)
                else:
                    AKI_3_12h_list.append(0)
                day_id_list.appens
                stay_id_list_2.append(wind//2)
            dict_ = {'stay_id':stay_id_list_2, 'icu_day_id':days, 'wind_12h':wind_12h, 'AKI_1_urine':AKI_1_list, 'AKI_2_urine':AKI_2_list, 'AKI_3_urine':AKI_3_list}
        
        return dict_
    except:
        print(stay_id)
        raise
        # errors.append(stay)

### Check

These cells are used for checking the function and its output:

In [ ]:
adm_urine_output_24h[adm_urine_output_24h.stay_id==stay_id]

In [ ]:
adm_urine_output_12h[adm_urine_output_12h.stay_id==stay_id]

In [ ]:
adm_urine_output_6h[adm_urine_output_6h.stay_id==stay_id]

In [ ]:
urine_output_df[urine_output_df.stay_id==stay_id].sort_values('charttime')

In [ ]:
stay_id=38118624
adm_urine_output_6h = sum_urine_output_6h[sum_urine_output_6h.stay_id==stay_id]
adm_urine_output_12h = sum_urine_output_12h[sum_urine_output_12h.stay_id==stay_id]
adm_urine_output_24h = sum_urine_output_24h[sum_urine_output_24h.stay_id==stay_id]
AKI_1_list = []
AKI_2_list = []
AKI_3_list = []
AKI_1_12h_list = []
AKI_2_12h_list = []
AKI_3_12h_list = []
stay_id_list_1 = []
stay_id_list_2 = []

days = list(adm_urine_output_24h.icu_day_id.unique())
arranged_days = np.arange(np.min(days), np.max(days) + 1)
for day in arranged_days:
    if day in days:
        # i = arranged_days.index(day)
        if sum(adm_urine_output_6h[adm_urine_output_6h.icu_day_id==day].AKI_1.values)>0:
            AKI_1_list.append(1)
        else:
            AKI_1_list.append(0)
        if (sum(adm_urine_output_12h[adm_urine_output_12h.icu_day_id==day].AKI_2.values)>0)&(AKI_1_list[day]==True):
            AKI_2_list.append(1)
        else:
            AKI_2_list.append(0)
        if (sum(adm_urine_output_24h[adm_urine_output_24h.icu_day_id==day].AKI_3.values)>0)&(AKI_1_list[day]==True):
            AKI_3_list.append(1)
        else:
            AKI_3_list.append(0)
    else:
        AKI_1_list.append(0)
        AKI_2_list.append(0)
        AKI_3_list.append(0)

In [ ]:
pd.DataFrame(get_aki_status_urine_out(38118624))

In [ ]:
patients = data_icustays.drop_duplicates(subset=['stay_id'], keep='first').subject_id.unique()
admissions = data_icustays.drop_duplicates(subset=['stay_id'], keep='first').hadm_id.unique()
icu_stays = data_icustays.drop_duplicates(subset=['stay_id'], keep='first').stay_id.unique()
icu_stays_weight_available = sum_urine_output_24h.stay_id.unique()
print('patients: ', len(patients))
print('admissions: ', len(admissions))
print("icu_stays: ", len(icu_stays))
print("icu stays with weight data available: ", len(icu_stays_weight_available))

### Multiprocessing 

Use multiptoccessing module for faster preprocessing:

In [ ]:
%%time
import traceback
import multiprocessing

stays = icu_stays_weight_available
pid_aki_status_urine_icu = pd.DataFrame()
chunk_size = 10000
# chunk_size = 10
n_chunks = len(stays)//chunk_size
# n_chunks = 2
errors = []

for i in range(0,n_chunks+1):
    with multiprocessing.Pool(processes=70) as pool_obj:
        pid_aki_status = pool_obj.map(get_aki_status_urine_out, stays[i*chunk_size:(i+1)*chunk_size])

    for e in pid_aki_status[:]:
        temp = pd.DataFrame(e)
        pid_aki_status_urine_icu = pd.concat([pid_aki_status_urine_icu, temp], ignore_index=True)

    print("Finished {} stays".format((i+1)*chunk_size))

    with open(PKL_PATH + 'pid_aki_status_urine_icu.pkl', 'wb') as f:
        pickle.dump(pid_aki_status_urine_icu, f, protocol=pickle.HIGHEST_PROTOCOL)

Check the result:

## Creatinine AKI criteria

Algorithm for detecting the AKI:

In [ ]:
data_icustays.describe()

In [ ]:
# function for calculating AKI status for each day of an admission
print_on = False
def get_aki_status(admission):
    try:
        adm_labs = labevents_creatinine[(labevents_creatinine.hadm_id==admission)].sort_values('day_id')
        if len(adm_labs)<1:
            print('no Scr values available for', admission)
            return
        patient = data_admissions[data_admissions.hadm_id==admission].subject_id.values[0]
        # admittime = data_admissions[data_admissions.hadm_id==admission].admittime
        days = adm_labs.day_id.values.tolist()
        hadm_ids = []
        aki_status = {'AKI_1':[], 'AKI_2':[], 'AKI_3':[]}
        AKI_1 = False
        AKI_2 = False
        AKI_3 = False
        specimen_ids = []

        all_days = np.arange(np.min(days), np.max(days)+1)

        for day in all_days:
            hadm_ids.append(admission)
            if day not in days:
                if print_on:
                    print(f'day {day} - no SCr')
                    print(f'AKI_1  [{int(False)}], AKI_2 [{int(False)}], AKI_3 [{int(False)}]')
                    print('----------------')
                aki_status['AKI_1'].append(int(AKI_1))
                aki_status['AKI_2'].append(int(AKI_2))
                aki_status['AKI_3'].append(int(AKI_3))
                specimen_ids.append(None)
                continue
            
            row_creatinine = adm_labs[adm_labs.day_id==day].sort_values('valuenum', ascending=False)
            C1 = float(row_creatinine.valuenum.values[0])
            charttime = row_creatinine.charttime.values[0]
            specimen_id = row_creatinine.specimen_id.values[0]
            
            if print_on:
                print(f'day {day} creatinine {C1}')
            
            df_baseline_1 = adm_labs[(adm_labs.charttime + np.timedelta64(24*7, 'h') >= charttime) \
                                            & (adm_labs.charttime < charttime)]

            df_median_baseline = labevents_creatinine[(labevents_creatinine.subject_id==patient)\
                                            &(labevents_creatinine.charttime + np.timedelta64(365, 'D') >= charttime) \
                                                & (labevents_creatinine.charttime < charttime)]

            df_baseline_2 = adm_labs[(adm_labs.charttime + np.timedelta64(48, 'h') >= charttime) \
                                            & (adm_labs.charttime < charttime)]
            # get baselines values
            if len(df_baseline_1) > 0:
                scr_baseline_1 = np.min(df_baseline_1.valuenum.values)

            elif len(df_median_baseline):
                scr_baseline_1 = np.median(df_median_baseline.valuenum.values)
            else:
                scr_baseline_1 = None

            if len(df_baseline_2) > 0:
                scr_baseline_2 = np.min(df_baseline_2.valuenum.values) #.sort_values('value', ascending=True).value.values[0]
            else:
                scr_baseline_2 = None
            if print_on:
                # print(f' baseline1 = {scr_baseline_1}, baseline2 = {scr_baseline_2}')
                print('baseline_1', scr_baseline_1)
                print('baseline_2', scr_baseline_2)
            # calculate aki
            if C1 >= 4.0:
                AKI_3 = True
                AKI_2 = True
                AKI_1 = True
            elif (scr_baseline_1 is not None):
                RV = np.round(C1 / scr_baseline_1, 2)
                if print_on:
                    print(f'RV = {C1} / {scr_baseline_1} = {RV}')
                if (RV >= 1.5):
                    if RV >=3.0:
                        AKI_3 = True
                        AKI_2 = True
                        AKI_1 = True
                    elif 3.0 > RV >=2.0:
                        AKI_3 = False
                        AKI_2 = True
                        AKI_1 = True
                    elif 2.0 > RV >= 1.5:
                        AKI_3 = False
                        AKI_2 = False
                        AKI_1 = True
                elif  (scr_baseline_2 is not None):
                    if (C1 - scr_baseline_2 > 0.29):
                        AKI_3 = False
                        AKI_2 = False
                        AKI_1 = True
                    else:
                        AKI_3 = False
                        AKI_2 = False
                        AKI_1 = False
                else:
                    AKI_3 = False
                    AKI_2 = False
                    AKI_1 = False
            elif (scr_baseline_2 is not None):
                if (C1 - scr_baseline_2 > 0.29):    
                        AKI_1 = False
                        AKI_2 = False
                        AKI_1 = True
            else:
                AKI_1 = False
                AKI_2 = False
                AKI_3 = False

            specimen_ids.append(specimen_id)
           
            aki_status['AKI_1'].append(int(AKI_1))
            aki_status['AKI_2'].append(int(AKI_2))
            aki_status['AKI_3'].append(int(AKI_3))

            if print_on:
                print(f'AKI_1  [{int(AKI_1)}], AKI_2 [{int(AKI_2)}], AKI_3 [{int(AKI_3)}]')
                print('----------------')
    except:
        print(admission)
        # raise

    return {'hadm_id':hadm_ids, 'specimen_id':specimen_ids, 'day_id':all_days, 'AKI_1':aki_status['AKI_1'], 'AKI_2':aki_status['AKI_2'], 'AKI_3':aki_status['AKI_3']}

In [ ]:
data_icustays.head(1)

In [ ]:
labevents_creatinine.head(1)

In [ ]:
data_d_items.head(1)

In [ ]:
patients = data_icustays.drop_duplicates(subset=['stay_id'], keep='first').subject_id.unique()
admissions = data_icustays.drop_duplicates(subset=['stay_id'], keep='first').hadm_id.unique()
icu_stays = data_icustays.drop_duplicates(subset=['stay_id'], keep='first').stay_id.unique()
print('patients: ', len(patients))
print('admissions: ', len(admissions))
print("icu_stays: ", len(icu_stays))
print('n of unique ICU stays with length of stay at least than 2 days: : ', (data_icustays[data_icustays.los>=2].stay_id.unique().shape[0]))


In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    #return lst3
    return lst3

In [ ]:
scr_counts = labevents_creatinine.groupby('hadm_id').count().sort_values('subject_id')
admissions_with_2_scr_tests = scr_counts[scr_counts.subject_id > 1].reset_index().hadm_id.unique()
icu_admissions_with_2_scr_tests = intersection(admissions, admissions_with_2_scr_tests)
icu_admissions_with_2_scr_tests_and_2_days_in_icu = intersection(icu_admissions_with_2_scr_tests, list(data_icustays[data_icustays.los>=2].hadm_id.unique()))
print('n of admissions with at least 2 SCr measurements and at least 2 days in ICU: ', len(icu_admissions_with_2_scr_temps_and_2_days_in_icu))

In [ ]:
%%time
pid_aki_status = []
errors = []
i = 0
for adm in icu_admissions_with_2_scr_tests_and_2_days_in_icu:
    if i % 100==0:
        print(f'------ {i}/{len(icu_admissions_with_2_scr_temps_and_2_days_in_icu)} admissions finished ------')
    try:
        pid_aki_status.append(get_aki_status(adm))
    except:
        print(adm)
        errors.add(adm)
    i += 1

In [ ]:
%%time
import traceback
import multiprocessing

admissions = icu_admissions_with_2_scr_tests_and_2_days_in_icu
final_aki_status_icu = pd.DataFrame()
chunk_size = 5000
n_chunks = len(admissions)//chunk_size
errors = []

for i in range(0,n_chunks+1):
    with multiprocessing.Pool(processes=80) as pool_obj:
        pid_aki_status = pool_obj.map(get_aki_status, admissions[i*chunk_size:(i+1)*chunk_size])

    for e in pid_aki_status[:]:
        temp = pd.DataFrame(e)
        final_aki_status_icu = pd.concat([final_aki_status_icu, temp], ignore_index=True)

    print("Finished {} admissions".format((i+1)*chunk_size))

    with open(PKL_PATH + 'final_aki_status_icu.pkl', 'wb') as f:
        pickle.dump(final_aki_status_icu, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
final_aki_status_icu[final_aki_status_icu.AKI_3==1].sort_values(['hadm_id', 'day_id']).drop_duplicates('hadm_id', keep='first')

## Combine two criterias

Filter stays:

In [ ]:
data_icustays = data_icustays[data_icustays.los>=2].sort_values(['subject_id', 'hadm_id', 'stay_id'])
print(data_icustays.shape)
data_icustays.head(1)

Merge createnine AKI status with labevents to get icu_day_id:

In [ ]:
with open(PKL_PATH + 'final_aki_status_icu.pkl', 'rb') as f:
    final_aki_status_icu = pickle.load(f)
final_aki_status_icu.rename(columns={'AKI_1':'AKI_1_scr', 'AKI_2':'AKI_2_scr', 'AKI_3':'AKI_3_scr'}, inplace=True)

In [ ]:
final_aki_status_icu.head(1)

In [ ]:
labevents.head(1)

In [ ]:
# merge labevents with icu_stays to get icu stay column
L = labevents.merge(data_icustays[['subject_id', 'hadm_id',	'stay_id', 'intime', 'outtime']]).sort_values(['subject_id', 'hadm_id', 'charttime'])
# remove wrong rows with inconsistency between intime, outtime and charttime
L = L[(L.charttime > L.intime)&(L.charttime < L.outtime)]
# get day ids for starting from admittime and intime
L['day_id'] = L['charttime'] - L['admittime']
L['day_id'] = [d.days for d in L.loc[:, 'day_id']]
L['icu_day_id'] = L['charttime'] - L['intime']
L['icu_day_id'] = [d.days for d in L.loc[:, 'icu_day_id']]

In [ ]:
final_aki_status_icu = final_aki_status_icu.merge(L[['specimen_id','icu_day_id', 'charttime', 'stay_id']], how='inner', on=['specimen_id']).drop_duplicates()
print(f'Number of unique admissions: ', final_aki_status_icu.hadm_id.unique().shape[0])
print(f'Shape: ', final_aki_status_icu.shape)
final_aki_status_icu.head()

In [ ]:
# patients with multiple icu stays
data_icustays[data_icustays.duplicated(subset=['subject_id', 'hadm_id'])]

Merge urine AKI status with icu_stays to get hadm_id:

In [ ]:
with open(PKL_PATH + 'pid_aki_status_urine_icu.pkl', 'rb') as f:
    pid_aki_status_urine_icu = pickle.load(f)

pid_aki_status_urine_icu.rename(columns={'day_id':'icu_day_id'}, inplace=True)
pid_aki_status_urine_icu = pid_aki_status_urine_icu.merge(data_icustays[['hadm_id', 'stay_id']], on='stay_id', how='left')

print(f'Number of unique ICU stays: ', pid_aki_status_urine_icu.stay_id.unique().shape[0])
print(f'Number of unique admissions: ', pid_aki_status_urine_icu.hadm_id.unique().shape[0])
print(f'Shape: ', pid_aki_status_urine_icu.shape)
pid_aki_status_urine_icu.head()

Merge two criterias on icu_day_id:

In [ ]:
aki_status_icu = final_aki_status_icu.merge(pid_aki_status_urine_icu, on=['hadm_id', 'icu_day_id'], how='left').sort_values(['hadm_id', 'day_id', 'icu_day_id'])
aki_status_icu

Filter out rows without stay_id:

In [ ]:
aki_status_icu = aki_status_icu[~(aki_status_icu.stay_id_x.isna())].rename(columns={'stay_id_x':'stay_id'}).drop(columns=['stay_id_y'])

In [ ]:
aki_status_icu.isna().sum()

Add urine criteria to SCr criteria:

In [ ]:
aki_status_icu = aki_status_icu.fillna(0)

In [ ]:
aki_status_icu['AKI_1'] = (aki_status_icu['AKI_1_scr'] + aki_status_icu['AKI_1_urine']).astype(bool).astype(int)
aki_status_icu['AKI_2'] = (aki_status_icu['AKI_2_scr'] + aki_status_icu['AKI_2_urine']).astype(bool).astype(int)
aki_status_icu['AKI_3'] = (aki_status_icu['AKI_3_scr'] + aki_status_icu['AKI_3_urine']).astype(bool).astype(int)

In [ ]:
print('Rows with positive AKI: ')
print('AKI_1 SCr:', aki_status_icu[aki_status_icu.AKI_1_scr==1].shape[0], 'Urine: ', aki_status_icu[aki_status_icu.AKI_1_urine==1].shape[0], 'Both: ', aki_status_icu[aki_status_icu.AKI_1==1].shape[0])
print('AKI_2 SCr:', aki_status_icu[aki_status_icu.AKI_2_scr==1].shape[0], 'Urine: ', aki_status_icu[aki_status_icu.AKI_2_urine==1].shape[0], 'Both: ', aki_status_icu[aki_status_icu.AKI_2==1].shape[0])
print('AKI_3 SCr:', aki_status_icu[aki_status_icu.AKI_3_scr==1].shape[0], 'Urine: ', aki_status_icu[aki_status_icu.AKI_3_scr==1].shape[0], 'Both: ', aki_status_icu[aki_status_icu.AKI_3==1].shape[0])

In [ ]:
print('ICU Stays with positive AKI: ')
print('AKI_1 SCr:', aki_status_icu[aki_status_icu.AKI_1_scr==1].stay_id.unique().shape[0], 'Urine: ', aki_status_icu[aki_status_icu.AKI_1_urine==1].stay_id.unique().shape[0], 'Both: ', aki_status_icu[aki_status_icu.AKI_1==1].stay_id.unique().shape[0])
print('AKI_2 SCr:', aki_status_icu[aki_status_icu.AKI_2_scr==1].stay_id.unique().shape[0], 'Urine: ', aki_status_icu[aki_status_icu.AKI_2_urine==1].stay_id.unique().shape[0], 'Both: ', aki_status_icu[aki_status_icu.AKI_2==1].stay_id.unique().shape[0])
print('AKI_3 SCr:', aki_status_icu[aki_status_icu.AKI_3_scr==1].stay_id.unique().shape[0], 'Urine: ', aki_status_icu[aki_status_icu.AKI_3_scr==1].stay_id.unique().shape[0], 'Both: ', aki_status_icu[aki_status_icu.AKI_3==1].stay_id.unique().shape[0])

In [ ]:
aki_status_icu = aki_status_icu.merge(data_icustays[['subject_id', 'hadm_id', 'stay_id', 'intime']])

In [ ]:
aki_status_icu.isna().sum()

In [ ]:
aki_status_icu['icu_day_id'] = aki_status_icu['charttime'] - aki_status_icu['intime']
aki_status_icu['icu_day_id'] = [d.days for d in aki_status_icu.loc[:, 'icu_day_id']]

aki_status_icu['icu_12h_window_id'] = aki_status_icu['charttime'] - aki_status_icu['intime']
aki_status_icu['icu_12h_window_id'] = [(d.days*24 + d.seconds//3600)//12 for d in aki_status_icu.loc[:, 'icu_12h_window_id']]

In [ ]:
# aki_status_icu = aki_status_icu[['subject_id', 'hadm_id', 'stay_id', 'specimen_id', 'day_id', 'icu_day_id', 'icu_12h_window_id', 'charttime', 'intime', 'AKI_1_scr',	'AKI_2_scr',	'AKI_3_scr', 'AKI_1_urine',	'AKI_2_urine',	'AKI_3_urine',	'AKI_1',	'AKI_2',	'AKI_3']]
aki_status_icu = aki_status_icu[['subject_id', 'hadm_id', 'stay_id', 'specimen_id', 'day_id', 'icu_day_id',  'charttime', 'intime', 'AKI_1_scr',	'AKI_2_scr',	'AKI_3_scr', 'AKI_1_urine',	'AKI_2_urine',	'AKI_3_urine',	'AKI_1',	'AKI_2',	'AKI_3']]

In [ ]:
aki_status_icu.head()

Add labels for model that uses softmax activation function:

In [ ]:
aki_stage_labels = aki_status_icu.copy()

In [ ]:
aki_stage_labels['AKI_2'] = list(np.where((aki_stage_labels['AKI_3']==0), aki_stage_labels['AKI_2'], 0))
aki_stage_labels['AKI_1'] = list(np.where(((aki_stage_labels['AKI_3']==0)&(aki_stage_labels['AKI_2']==0)), aki_stage_labels['AKI_1'], 0))
aki_stage_labels['ANY_AKI'] = np.where(((icu_aki_status['AKI_1']==0)&(icu_aki_status['AKI_2']==0)&(icu_aki_status['AKI_3']==0)), 0, 1)
aki_stage_labels['NO_AKI'] = np.where(((icu_aki_status['AKI_1']==0)&(icu_aki_status['AKI_2']==0)&(icu_aki_status['AKI_3']==0)), 1, 0)

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'aki_stage_labels.pkl', 'wb') as f:
    pickle.dump(aki_stage_labels, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'aki_status_icu.pkl', 'wb') as f:
    pickle.dump(aki_status_icu, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'aki_status_icu.pkl', 'rb') as f:
    aki_status_icu = pickle.load(f)

In [ ]:
with open(PKL_PATH + 'aki_stage_labels.pkl', 'rb') as f:
    aki_stage_labels = pickle.load(f)

In [ ]:
aki_status_icu.stay_id.unique().shape

# Filtering admissions and stays

### Filter out admissions with AKI diagnoses and no AKI found by criterias:

In [ ]:
# icd codes for AKI
AKI_icd = list(data_d_icd_diagnoses[(data_d_icd_diagnoses.long_title.str.contains('acute'))\
                & (data_d_icd_diagnoses.long_title.str.contains('kidney'))\
                    & (data_d_icd_diagnoses.long_title.str.contains(('|').join(['failure', 'injury'])))\
                        |(data_d_icd_diagnoses.icd_code.str.contains('N17'))\
                            |(data_d_icd_diagnoses.icd_code.str.contains('^584'))]\
                        .icd_code.values)

print(AKI_icd)

In [ ]:
data_d_icd_diagnoses.head(1)

In [ ]:
for _, row in data_d_icd_diagnoses[data_d_icd_diagnoses.icd_code.isin(AKI_icd)].iterrows():
    print(row.icd_code, row.icd_version, row.long_title)
    # print(row)

In [ ]:
# admissions with AKI icd code 
aki_admissions = list(data_diagnoses[data_diagnoses.icd_code.isin(AKI_icd)].hadm_id.unique())
print(len(aki_admissions))


In [ ]:
g = aki_status_icu.groupby('hadm_id')
g = pd.DataFrame(g.sum().AKI_1==0).reset_index()
neg_by_criteria_adms = g[g.AKI_1==True].hadm_id.unique()
print(f'n admissions with no AKI found by criterias: ', len(neg_by_criteria_adms))

pos_by_criteris_adms = aki_status_icu[(aki_status_icu.AKI_1==1)].hadm_id.unique()
print(f'n admissions with AKI found by criterias: ', len(pos_by_criteris_adms))

print(f'total adms with AKI status available: ', aki_status_icu.hadm_id.unique().shape[0])
print(f'total stays with AKI status available: ', aki_status_icu.stay_id.unique().shape[0])

In [ ]:
fn_adms = aki_status_icu[(aki_status_icu.hadm_id.isin(aki_admissions))&(aki_status_icu.hadm_id.isin(neg_by_criteria_adms))].hadm_id.unique()
fn_stays = aki_status_icu[(aki_status_icu.hadm_id.isin(aki_admissions))&(aki_status_icu.hadm_id.isin(neg_by_criteria_adms))].stay_id.unique()
print(f'(FN) admissions with AKI diagnoses and no AKI found by criterias: ',fn_adms.shape[0], ', stays: ', fn_stays.shape[0])
tn_adms = aki_status_icu[(~(aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(neg_by_criteria_adms))].hadm_id.unique()
tn_stays = aki_status_icu[(~(aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(neg_by_criteria_adms))].stay_id.unique()
print(f'(TN) admissions with no AKI diagnoses and no AKI found by criterias: ',tn_adms.shape[0], ', stays: ', tn_stays.shape[0])

fp_adms = aki_status_icu[(~(aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(pos_by_criteris_adms))].hadm_id.unique()
fp_stays = aki_status_icu[(~(aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(pos_by_criteris_adms))].stay_id.unique()
print(f'(FP) admissions with no AKI diagnoses and AKI found by criterias: ',fp_adms.shape[0], ', stays: ', fp_stays.shape[0])
tp_adms = aki_status_icu[((aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(pos_by_criteris_adms))].hadm_id.unique()
tp_stays = aki_status_icu[((aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(pos_by_criteris_adms))].stay_id.unique()
print(f'(TP) admissions with  AKI diagnoses and AKI found by criterias: ',tp_adms.shape[0], ', stays: ', tp_stays.shape[0])

print(f'total admissions: {len(fn_adms)+len(tn_adms)+len(fp_adms)+len(tp_adms)}, stays: {len(fn_stays)+len(tn_stays)+len(fp_stays)+len(tp_stays)}')

### Abnormal createnine on the fisrt day:

First day of an admission:

In [ ]:
F = labevents_creatinine.sort_values(['subject_id', 'hadm_id', 'charttime'], ascending=True).drop_duplicates('hadm_id', keep='first')
normal_first_scr_adms = list(F[F.flag==''].hadm_id.unique())
len(normal_first_scr_adms)

In [ ]:
data_icustays.head(3)

In [ ]:
labevents_creatinine[labevents_creatinine.hadm_id==24597018].sort_values(['subject_id', 'hadm_id', 'charttime'], ascending=True)

In [ ]:
# Stays and admissions with available AKI status and first SCr test result - normal
normal_scr_adms = aki_status_icu[aki_status_icu.hadm_id.isin(normal_first_scr_adms)].hadm_id.unique()
normal_scr_stays = aki_status_icu[aki_status_icu.hadm_id.isin(normal_first_scr_adms)].stay_id.unique()

In [ ]:
print(f'\nn admissions with available AKI status and first creatinine measurement==normal: ', len(normal_scr_adms))
print(f'n ICU stays with available AKI status and first creatinine measurement==normal: ', len(normal_scr_stays))

First day of ICU stay

In [ ]:
fisrt_scr_icu = chartevents_creatinine.sort_values(['subject_id','hadm_id','stay_id','charttime'], ascending=True).drop_duplicates(['subject_id','hadm_id','stay_id'], keep='first')

In [ ]:
stays_fisrt_scr_normal_icu = fisrt_scr_icu[~(fisrt_scr_icu.valuenum>1.2)].stay_id.unique()
adms_fisrt_scr_normal_icu = fisrt_scr_icu[~(fisrt_scr_icu.valuenum>1.2)].hadm_id.unique()

# Stays and admissions with available AKI status and first SCr test result - normal
normal_scr_adms = aki_status_icu[aki_status_icu.stay_id.isin(stays_fisrt_scr_normal_icu)].hadm_id.unique()
normal_scr_stays = aki_status_icu[aki_status_icu.stay_id.isin(stays_fisrt_scr_normal_icu)].stay_id.unique()

In [ ]:
print(f'n admissions with available AKI status: ', aki_status_icu.hadm_id.unique().shape[0])
print(f'n ICU stays with available AKI status: ', aki_status_icu.stay_id.unique().shape[0])

print(f'n ICU stays with available AKI status and first creatinine measurement (during ICU stay)==normal: ', len(normal_scr_stays))
# print(f'n admissions with available AKI status and first creatinine measurement (during ICU stay)==normal: ', len(adms_fisrt_scr_normal_icu))

Describing the labels:

In [ ]:
aki_status_icu.head(1)

In [ ]:
fn_adms = aki_status_icu[(aki_status_icu.stay_id.isin(normal_scr_stays))&(aki_status_icu.hadm_id.isin(aki_admissions))&(aki_status_icu.hadm_id.isin(neg_by_criteria_adms))].hadm_id.unique()
fn_stays = aki_status_icu[(aki_status_icu.stay_id.isin(normal_scr_stays))&(aki_status_icu.hadm_id.isin(aki_admissions))&(aki_status_icu.hadm_id.isin(neg_by_criteria_adms))].stay_id.unique()
print(f'(FN) admissions with AKI diagnoses and no AKI found by criterias: ',fn_adms.shape[0], ', stays --->', fn_stays.shape[0])
tn_adms = aki_status_icu[(aki_status_icu.stay_id.isin(normal_scr_stays))&(~(aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(neg_by_criteria_adms))].hadm_id.unique()
tn_stays = aki_status_icu[(aki_status_icu.stay_id.isin(stays_fisrt_scr_normal_icu))&(~(aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(neg_by_criteria_adms))].stay_id.unique()
print(f'(TN) admissions with no AKI diagnoses and no AKI found by criterias: ',tn_adms.shape[0], ', stays ---> ', tn_stays.shape[0])

fp_adms = aki_status_icu[(aki_status_icu.stay_id.isin(normal_scr_stays))&(~(aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(pos_by_criteris_adms))].hadm_id.unique()
fp_stays = aki_status_icu[(aki_status_icu.stay_id.isin(normal_scr_stays))&(~(aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(pos_by_criteris_adms))].stay_id.unique()
print(f'(FP) admissions with no AKI diagnoses and AKI found by criterias: ',fp_adms.shape[0], ', stays ---> ', fp_stays.shape[0])
tp_adms = aki_status_icu[(aki_status_icu.stay_id.isin(normal_scr_stays))&((aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(pos_by_criteris_adms))].hadm_id.unique()
tp_stays = aki_status_icu[(aki_status_icu.stay_id.isin(normal_scr_stays))&((aki_status_icu.hadm_id.isin(aki_admissions)))&(aki_status_icu.hadm_id.isin(pos_by_criteris_adms))].stay_id.unique()
print(f'(TP) admissions with  AKI diagnoses and AKI found by criterias: ',tp_adms.shape[0], ', stays ---> ', tp_stays.shape[0])

print(f'total admissions: {len(fn_adms)+len(tn_adms)+len(fp_adms)+len(tp_adms)}, stays ---> {len(fn_stays)+len(tn_stays)+len(fp_stays)+len(tp_stays)}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories, annot_kws={"size": 18})

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

tn, fp, fn, tp = 10676, 8530, 982, 3784
cf = np.asarray([tn, fp, fn, tp]).reshape(2,2)

labels = ['TN','FP', 'FN', 'TP']
# labels = np.asarray(labels).reshape(2,2)
# categories = ['Algorithm', 'Diagnoses records']

make_confusion_matrix(cf, 
                      group_names=labels,
                      categories=False, 
                      xyticks=True,
                      xyplotlabels=False,
                      sum_stats=False,
                      percent=False,
                      cbar=False
                      )
plt.ylabel('Diagnosis            No diagnosis', fontsize=12)
plt.xlabel('No AKI found by algorithm               AKI found by algorithm', fontsize=10)

Filter out stays with AKI onset on the first day in ICU. I kept AKI stage 1 cases.

In [ ]:
aki_status_icu = aki_status_icu.sort_values(['subject_id',	'hadm_id',	'stay_id', 'icu_day_id'])
F = aki_status_icu.drop_duplicates('stay_id', keep='first')
F = F[(~(F.AKI_2==1))&((F.stay_id.isin(tp_stays))|(F.stay_id.isin(tn_stays)))]

In [ ]:
stays_tp_tn_normalSCr = F.stay_id.to_list()
print(len(stays_tp_tn_normalSCr))

In [ ]:
# all admissions are at least 2 days long
data_icustays[(data_icustays.stay_id.isin(stays_tp_tn_normalSCr))&(data_icustays.los<2)]

In [ ]:
# double check if all stays have info for the first 2 days in icu
stays_tp_tn_normalSCr_have_01_days = data_icustays[(data_icustays.stay_id.isin(stays_tp_tn_normalSCr))&~(data_icustays.stay_id.isin(stays_without_12h_0123))].stay_id.unique()
print(len(stays_tp_tn_normalSCr_have_01_days))

Splitting adms

In [ ]:
from sklearn.model_selection import train_test_split

train_stays, temp = train_test_split(stays_tp_tn_normalSCr_have_01_days, test_size=0.2, random_state=42)
test_stays, val_stays = train_test_split(temp, test_size=0.5, random_state=42)

In [ ]:
print(len(train_stays))
print(len(test_stays))
print(len(val_stays))

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    #return lst3
    return lst3

In [ ]:
len(intersection(train_stays, tp_stays))

In [ ]:
len(intersection(test_stays, tp_stays))

### Checking

In [ ]:
adm = tp_adms[0]
aki_status_icu[aki_status_icu.hadm_id==adm]

In [ ]:
# D = data_diagnoses.merge(data_d_icd_diagnoses)
subject_id = D[D.hadm_id==adm].subject_id.values[0]
D[D.hadm_id==adm]

In [ ]:
data_icustays[data_icustays.subject_id==subject_id]

In [ ]:
sum_urine_output_6h[sum_urine_output_6h.hadm_id==adm]

In [ ]:
sum_urine_output_24h[sum_urine_output_24h.hadm_id==adm]

In [ ]:
# weight_data[weight_data.subject_id==subject_id]
weight_data_icu[weight_data_icu.subject_id==subject_id]

In [ ]:
urine_output_df[urine_output_df.hadm_id==adm]

In [ ]:
# labevents_creatinine[labevents_creatinine.hadm_id==22595853]
labevents_creatinine[labevents_creatinine.subject_id==subject_id]

# Chartevents vitals

## Making pid_vitals dataset

### Preparing data

In [ ]:
chart_items_vitals = [225309, 225312, 226329, 220048, 225310, 224192, 228640, 220179, 223762, 227243, 220181, 224643, 224167, 220052, 220051, 220050, 220180,
                        227242, 223761, 223765, 223766, 224645, 224646, 224647, 226092, 220045, 224359, 227016, 227015, 220046, 220047, 227858, 227916, 223764,
                        223765, 224647]
print(len(set(chart_items_vitals)))
for _, row in data_d_items[data_d_items.itemid.isin(chart_items_vitals)].iterrows():
    print(row.itemid, '|', row.label, '|', row.category,)

In [ ]:
# remove lab events from the table, because they are in labevents already 
data_chartevents_nl = data_chartevents[~data_chartevents.category.str.contains('Labs')]

# create dataframe with unique item ids
unique_chart_items = data_chartevents_nl.drop_duplicates(subset=['itemid'])

chartevents_vitals = data_chartevents_nl[data_chartevents_nl.category=='Routine Vital Signs']
print(chartevents_vitals.shape)

In [ ]:
# format_ = '%Y-%m-%d %H:%M:%S'
# chartevents_vitals['charttime'] = pd.to_datetime(chartevents_vitals['charttime'], format=format_)

chartevents_vitals['valueuom'] = chartevents_vitals['valueuom'].fillna(-1).astype(float)

In [ ]:
chartevents_vitals = chartevents_vitals.merge(data_icustays[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime']])\
                                        .merge(data_admissions[['subject_id', 'hadm_id', 'admittime']])

In [ ]:
chartevents_vitals['icu_day_id'] = chartevents_vitals['charttime'] - chartevents_vitals['intime']
chartevents_vitals['icu_day_id'] = [d.days for d in chartevents_vitals.loc[:, 'icu_day_id']]

chartevents_vitals['icu_12h_window_id'] = chartevents_vitals['charttime'] - chartevents_vitals['intime']
chartevents_vitals['icu_12h_window_id'] = [(d.days*24 + d.seconds//3600)//12 for d in chartevents_vitals.loc[:, 'icu_12h_window_id']]

In [ ]:
chartevents_vitals['day_id'] = chartevents_vitals['charttime'] - chartevents_vitals['admittime']
chartevents_vitals['day_id'] = [d.days for d in chartevents_vitals.loc[:, 'day_id']]

In [ ]:
chartevents_vitals = chartevents_vitals.sort_values(['subject_id', 'charttime', 'stay_id'])

In [ ]:
str_chartevents_vitals = chartevents_vitals[chartevents_vitals.value.str.contains('[a-zA-Z]')]
str_chartevents_vitals.abbreviation.unique()

In [ ]:
num_chartevents_vitals = chartevents_vitals[~(chartevents_vitals.value.str.contains('[a-zA-Z]'))]
num_chartevents_vitals['valuenum'] = num_chartevents_vitals['valuenum'].astype(float)

In [ ]:
chartevents_vitals.head(5)

In [ ]:
chartevents_vitals.abbreviation.unique()

In [ ]:
abbreviation_mapping = {'Orthostatic HR lying':'ohr lying', 'Orthostatic HR sitting':'ohr sitting', 'Orthostatic BPs lying':'obps lying', \
    'Orthostatic BPd lying':'obpd lying', 'Orthostatic BPs sitting':'obps sitting', 'Orthostatic BPs standing':'obps standing', \
       'Orthostatic HR standing':'ohr standing',  'Rest HR -  Aerobic Activity Response':'rhr aar'}

In [ ]:
chartevents_vitals.replace({'abbreviation':abbreviation_mapping}, inplace=True)

In [ ]:
chartevents_vitals.abbreviation.unique()

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'chartevents_vitals.pkl', 'wb') as f:
    pickle.dump(chartevents_vitals, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'chartevents_vitals.pkl', 'rb') as f:
    chartevents_vitals = pickle.load(f)

### Making PID for 12h window

In [ ]:
# number of admissions with aki status available from ICU wards
admissions = aki_status_icu.hadm_id.unique()
stays = aki_status_icu.stay_id.unique()
print(admissions.shape[0])
print(stays.shape[0])

Test

In [ ]:
stay = stays[0]
data_icustays[data_icustays.stay_id==stay]

In [ ]:
chartevents_vitals.head(1)

In [ ]:
test = num_chartevents_vitals[num_chartevents_vitals.stay_id.isin(stays[:3])][['subject_id', 'hadm_id', 'stay_id', 'valuenum', 'valueuom', 'label', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id','label', 'icu_day_id', 'icu_12h_window_id'])\
        .agg({'valueuom':min, 'valuenum':['min', 'max']})\
            .reset_index()
test

In [ ]:
test = test.droplevel(0, axis=1)
test.columns = [ 'subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id', 'icu_12h_window_id', 'valueuom', 'min_value', 'max_value']
test = test.sort_values(['subject_id', 'icu_12h_window_id'])
test['vitals'] = test['label'] + ' {' + test['min_value'].astype(str) + '} ' + ' {' + test['max_value'].astype(str) + '} ' + test['valueuom']
test.groupby(['icu_12h_window_id','icu_day_id', 'subject_id', 'hadm_id', 'stay_id']).vitals.apply('; '.join).reset_index(name='vitals')
test

Implementation:

In [ ]:
num_chartevents_vitals.info()

In [ ]:
# tried to use label OR abbrevation OR itemid
temp = num_chartevents_vitals[['subject_id', 'hadm_id', 'stay_id', 'itemid', 'valuenum', 'valueuom', 'abbreviation', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id','itemid', 'icu_day_id', 'icu_12h_window_id'])\
        .agg({'valueuom':min, 'valuenum':['min', 'max']})\
            .reset_index()

In [ ]:
temp.head(1)

In [ ]:
# %%time
# print('1st row im progress..')
# temp = temp.droplevel(0, axis=1)
# print('2st row im progress..')
# temp.columns = [ 'subject_id', 'hadm_id', 'stay_id', 'itemid', 'icu_day_id', 'icu_12h_window_id', 'valueuom', 'min_value', 'max_value']
# print('3st row im progress..')
# temp = temp.sort_values(['subject_id', 'icu_12h_window_id'])
print('4st row im progress..')
temp['vitals_codes'] = 'v'+ temp['itemid'].astype(str) + ' {' + temp['min_value'].astype(str) + '} ' + ' {' + temp['max_value'].astype(str) + '}'
print('5st row im progress..')
pid_vitals_codes = temp.groupby(['icu_12h_window_id','icu_day_id', 'subject_id', 'hadm_id', 'stay_id']).vitals_codes.apply('; '.join).reset_index(name='vitals_codes').sort_values(['subject_id', 'icu_12h_window_id'])

In [ ]:
pid_vitals_codes.head(1)

In [ ]:
print(pid_vitals.hadm_id.unique().shape)
pid_vitals.head(1)

In [ ]:
pid_vitals = pid_vitals.merge(pid_vitals_codes, on=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id', 'icu_12h_window_id'], how='left')

In [ ]:
pid_vitals = pid_vitals.drop(columns=['vitals_codes_x']).rename(columns={'vitals_codes_y':'vitals_codes'})
pid_vitals.head(1)

In [ ]:
pid_vitals.iloc[1,:].vitals

In [ ]:
pid_vitals.iloc[1,:].vitals_codes

 Save

In [ ]:
with open(PKL_PATH + 'pid_vitals_icu.pkl', 'wb') as f:
    pickle.dump(pid_vitals, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'pid_vitals_icu.pkl', 'rb') as f:
    pid_vitals = pickle.load(f)

### Check

In [ ]:
pid_vitals[pid_vitals.stay_id==37510196]

In [ ]:
chartevents_vitals[(chartevents_vitals.stay_id==37510196)&(chartevents_vitals.label=='Non Invasive Blood Pressure systolic')&(chartevents_vitals.icu_12h_window_id==6)].sort_values('charttime')

In [ ]:
data_icustays[data_icustays.subject_id==10001884]

# Outputevents

In [ ]:
data_d_items[data_d_items.label.str.lower().str.contains('urine')]

In [ ]:
data_outputevents.head(1)

In [ ]:
data_outputevents.info()

We have only two categories in outputevents:

In [ ]:
print(data_outputevents.category.unique())

The most frequent output events are:

In [ ]:
data_outputevents.groupby(['label']).count().sort_values('subject_id', ascending=False)

Print unique labels for outputs:

In [ ]:
# print items
unique_output_items = data_outputevents.drop_duplicates('label')
print('itemid | label | category | valuenum')
for _, row in  unique_output_items.sort_values('category').iterrows():
    print(row.itemid, '|', row.label, '|', row.category, '|', row.value)

## Get all outputs:

### Test:

In [ ]:
test = data_outputevents[data_outputevents.stay_id.isin(stays[:3])][['subject_id', 'hadm_id', 'stay_id', 'value', 'valueuom', 'itemid', 'label', 'abbreviation', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id','itemid','label', 'abbreviation', 'icu_day_id', 'icu_12h_window_id'])\
        .agg({'valueuom':min, 'value':sum})\
            .reset_index()
test.head()

In [ ]:
test.columns = [ 'subject_id', 'hadm_id', 'stay_id','itemid', 'label', 'abbreviation', 'icu_day_id', 'icu_12h_window_id', 'valueuom', 'sum_output']
test = test.sort_values(['subject_id', 'icu_12h_window_id'])
test['outputs_codes'] = 'o' + test['itemid'].astype(str) + ' {' + test['sum_output'].astype(str) + '}'
test = test.groupby(['icu_12h_window_id','icu_day_id', 'subject_id', 'hadm_id', 'stay_id']).outputs_codes.apply('; '.join).reset_index(name='outputs_codes').sort_values(['subject_id', 'icu_12h_window_id'])
test.head()

### Implementation

In [ ]:
%%time
g = data_outputevents[['subject_id', 'hadm_id', 'stay_id', 'value', 'valueuom', 'itemid', 'label', 'abbreviation', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id','itemid','label', 'abbreviation', 'icu_day_id', 'icu_12h_window_id'])\
        .agg({'valueuom':min, 'value':sum})\
            .reset_index()
g

In [ ]:
%%time
print(f'1st row is in progress...')
g.columns = [ 'subject_id', 'hadm_id', 'stay_id','itemid', 'label', 'abbreviation', 'icu_day_id', 'icu_12h_window_id', 'valueuom', 'sum_output']
g= g.sort_values(['subject_id', 'icu_12h_window_id'])
print(f'2nd row is in progress...')
g['outputs_codes'] = 'o' + g['itemid'].astype(str) + ' {' + g['sum_output'].astype(str) + '}'
print(f'3nd row is in progress...')
pid_outputs_codes = g.groupby(['icu_12h_window_id','icu_day_id', 'subject_id', 'hadm_id', 'stay_id']).outputs_codes.apply('; '.join)\
                        .reset_index(name='outputs_codes').sort_values(['subject_id', 'icu_12h_window_id'])
pid_outputs_codes

In [ ]:
pid_outputs = pid_outputs.merge(pid_outputs_codes, on=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id', 'icu_12h_window_id'], how='left')

In [ ]:
pid_outputs.head()

Save pid_outputs:

In [ ]:
with open(PKL_PATH + 'pid_outputs.pkl', 'wb') as f:
    pickle.dump(pid_outputs, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'pid_outputs.pkl', 'rb') as f:
    pid_outputs = pickle.load(f)

### Check:

In [ ]:
data_icustays[data_icustays.subject_id==19999987]

In [ ]:
stay = 36195440
pid_outputs[pid_outputs.stay_id==stay].outputs.values[1]

In [ ]:
data_outputevents[(data_outputevents.stay_id==stay)&(data_outputevents.label=='Oral Gastric')&(data_outputevents.icu_12h_window_id==1)].sort_values('charttime')

# Medications

In [ ]:
data_inputevents.rateuom.unique()

In [ ]:
%%time
data_inputevents['length_minutes'] = data_inputevents['endtime'] - data_inputevents['starttime']
data_inputevents['length_minutes'] = [t.days*24*60 + t.seconds // 60 for t in data_inputevents.loc[:, 'length_minutes']]

In [ ]:
%%time
data_inputevents['start_icu_day_id'] = data_inputevents['starttime'] - data_inputevents['intime']
data_inputevents['start_icu_day_id'] = [t.days for t in data_inputevents.loc[:, 'start_icu_day_id']]

data_inputevents['stop_icu_day_id'] = data_inputevents['endtime'] - data_inputevents['intime']
data_inputevents['stop_icu_day_id'] = [t.days for t in data_inputevents.loc[:, 'stop_icu_day_id']]

In [ ]:
%%time
data_inputevents['start_icu_12h_id'] = data_inputevents['starttime'] - data_inputevents['intime']
data_inputevents['start_icu_12h_id'] = [(t.days*24 + t.seconds//3600)//12 for t in data_inputevents.loc[:, 'start_icu_12h_id']]

data_inputevents['stop_icu_12h_id'] = data_inputevents['endtime'] - data_inputevents['intime']
data_inputevents['stop_icu_12h_id'] = [(t.days*24 + t.seconds//3600)//12 for t in data_inputevents.loc[:, 'stop_icu_12h_id']]

In [ ]:
unique_rates = data_inputevents.drop_duplicates('rateuom')[['subject_id', 'hadm_id', 'stay_id', 'intime', 'starttime', 'endtime', 'length_minutes', 'itemid', 'amount', 'amountuom', 'rate',	'rateuom', 'patientweight', 'ordercategoryname']]
unique_rates

In [ ]:
data_inputevents.ordercategoryname.unique()

In [ ]:
data_inputevents.secondaryordercategoryname.unique()

In [ ]:
data_inputevents.ordercategorydescription.unique()

In [ ]:
print(data_inputevents.length_minutes.max())
data_inputevents.length_minutes.describe()

In [ ]:
rates = list(unique_rates.rateuom.values)
rates

In [ ]:
data_medications = data_inputevents[data_inputevents.category=='Medications']
data_medications = data_medications[['subject_id',	'hadm_id',	'stay_id',	'starttime',	'endtime', 'itemid',	'amount',	'amountuom', 'rate',	'rateuom', 'patientweight', 'intime', 'length_minutes',	'start_icu_day_id',	'stop_icu_day_id',	'start_icu_12h_id','stop_icu_12h_id','label']]
data_medications.head(2)

Save/load dataframe:

In [ ]:
with open(PKL_PATH+'data_medications_icu.pkl', 'wb') as f:
    pickle.dump(data_medications, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH+'data_medications_icu.pkl', 'rb') as f:
    data_medications = pickle.load(f)

### Test

In [ ]:
data_medications.head()

In [ ]:
data_icustays[data_icustays.stay_id==32336619]

In [ ]:
stay = 39553978
day = 2
wind = 0
stay_medications = data_medications[data_medications.stay_id==stay]
subject_id = stay_medications.subject_id.unique()[0]
hadm_id = stay_medications.hadm_id.unique()[0]
if len(stay_medications)<1:
    print('no meds')
min_day_id = np.min(stay_medications.start_icu_day_id)
max_day_id = np.max(stay_medications.stop_icu_day_id)
days_arranged = np.arange(min_day_id, max_day_id+1)

min_wind_id = np.min(stay_medications.start_icu_12h_id)
max_wind_id = np.max(stay_medications.stop_icu_12h_id)
winds_arranged = np.arange(min_wind_id, max_wind_id+1)

med_names_l = []
med_codes_l = []
subject_id_l = []
hadm_id_l = []
days_arranged = []

for wind in winds_arranged:
    day = wind // 2
    # day_med_info = stay_medications[(stay_medications.start_icu_day_id <= day)&(stay_medications.stop_icu_day_id >= day)]
    _12h_med_info = stay_medications[(stay_medications.start_icu_12h_id <= wind)&(stay_medications.stop_icu_12h_id >= wind)]
    med_names = _12h_med_info.label.unique()
    med_codes = _12h_med_info.itemid.unique()
    med_names_l.append('; '.join(med_names))
    med_codes_l.append(' '.join(map(str,med_codes)))
    subject_id_l.append(subject_id)
    hadm_id_l.append(hadm_id)
    days_arranged.append(day)

pid_medications = {'subject_id': subject_id_l, 'hadm_id': hadm_id_l, 'icu_day_id': days_arranged, '_12h_window_id':winds_arranged, 'medications_names': med_names_l, 'medications_codes':med_codes_l}


In [ ]:
for k,v in pid_medications.items():
    print(k, len(pid_medications[k]))

In [ ]:
pd.DataFrame(pid_medications)

In [ ]:
# function
def get_medications_data(stay, raise_=False, winds_arranged=[0,1]):
    try:
        stay_medications = data_medications[data_medications.stay_id==stay]
        if len(stay_medications)<1:
            return
        subject_id = stay_medications.subject_id.unique()[0]
        hadm_id = stay_medications.hadm_id.unique()[0]
        if len(stay_medications)<1:
            return
        min_day_id = np.min(stay_medications.start_icu_day_id)
        max_day_id = np.max(stay_medications.stop_icu_day_id)
        days_arranged = np.arange(min_day_id, max_day_id+1)
        if winds_arranged is None:
            min_wind_id = np.min(stay_medications.start_icu_12h_id)
            max_wind_id = np.max(stay_medications.stop_icu_12h_id)
            winds_arranged = np.arange(min_wind_id, max_wind_id+1)

        med_names_l = []
        med_codes_l = []
        subject_id_l = []
        hadm_id_l = []
        stay_id_l = []
        days_arranged = []

        for wind in winds_arranged:
            day = wind // 2
            # day_med_info = stay_medications[(stay_medications.start_icu_day_id <= day)&(stay_medications.stop_icu_day_id >= day)]
            _12h_med_info = stay_medications[(stay_medications.start_icu_12h_id <= wind)&(stay_medications.stop_icu_12h_id >= wind)]
            # med_names = _12h_med_info.label.unique()
            med_codes = list(_12h_med_info.itemid.unique())
            # med_names_l.append('; '.join(med_names))
            # med_codes_l.append(' '.join(map(lambda x: 'm' + str(x), med_codes)))
            med_codes_l = [*med_codes_l, *med_codes]
            subject_id_l.append(subject_id)
            hadm_id_l.append(hadm_id)
            stay_id_l.append(stay)
            days_arranged.append(day)

        pid_medications = {'subject_id': subject_id_l, 'hadm_id': hadm_id_l, 'stay_id':stay_id_l, 'icu_day_id': days_arranged, '_12h_window_id':winds_arranged, 'medications_codes':med_codes_l}
        # pid_medications = {'subject_id': subject_id_l, 'hadm_id': hadm_id_l, 'stay_id':stay_id_l, 'icu_day_id': days_arranged, '_12h_window_id':winds_arranged, 'medications_names': med_names_l, 'medications_codes':med_codes_l}
        return pid_medications
    except:
        print(stay)
        if raise_:
            raise
        return

In [ ]:
# function
def get_medications_data(stay, raise_=False, winds_arranged=[0,1]):
    try:
        stay_medications = data_medications[data_medications.stay_id==stay]
        if len(stay_medications)<1:
            return
        subject_id = stay_medications.subject_id.unique()[0]
        hadm_id = stay_medications.hadm_id.unique()[0]
        if len(stay_medications)<1:
            return
        min_day_id = np.min(stay_medications.start_icu_day_id)
        max_day_id = np.max(stay_medications.stop_icu_day_id)
        days_arranged = np.arange(min_day_id, max_day_id+1)
        if winds_arranged is None:
            min_wind_id = np.min(stay_medications.start_icu_12h_id)
            max_wind_id = np.max(stay_medications.stop_icu_12h_id)
            winds_arranged = np.arange(min_wind_id, max_wind_id+1)

        med_names_l = []
        med_codes_l = []
        subject_id_l = []
        hadm_id_l = []
        stay_id_l = []
        days_arranged = []

        for wind in winds_arranged:
            day = wind // 2
            # day_med_info = stay_medications[(stay_medications.start_icu_day_id <= day)&(stay_medications.stop_icu_day_id >= day)]
            _12h_med_info = stay_medications[(stay_medications.start_icu_12h_id <= wind)&(stay_medications.stop_icu_12h_id >= wind)]
            # med_names = _12h_med_info.label.unique()
            med_codes = list(_12h_med_info.itemid.unique())
            # med_names_l.append('; '.join(med_names))
            # med_codes_l.append(' '.join(map(lambda x: 'm' + str(x), med_codes)))
            med_codes_l = [*med_codes_l, *med_codes]
            subject_id_l.append(subject_id)
            hadm_id_l.append(hadm_id)
            stay_id_l.append(stay)
            days_arranged.append(day)

        pid_medications = {'subject_id': subject_id_l, 'hadm_id': hadm_id_l, 'stay_id':stay_id_l, 'icu_day_id': days_arranged, '_12h_window_id':winds_arranged, 'medications_codes':med_codes_l}
        # pid_medications = {'subject_id': subject_id_l, 'hadm_id': hadm_id_l, 'stay_id':stay_id_l, 'icu_day_id': days_arranged, '_12h_window_id':winds_arranged, 'medications_names': med_names_l, 'medications_codes':med_codes_l}
        return pid_medications
    except:
        print(stay)
        if raise_:
            raise
        return

In [ ]:
get_medications_data(stay, raise_=True)

In [ ]:
stay = 37510196
pd.DataFrame(get_medications_data(stay, raise_=True))

In [ ]:
data_medications[data_medications.stay_id==stay].sort_values(['start_icu_12h_id',	'stop_icu_12h_id'])

### Multiprocessing

In [ ]:
%%time
import traceback
import multiprocessing

final_meds = pd.DataFrame()
chunk_size = 15000
stays = list(data_icustays.stay_id.unique())
n_chunks = len(stays)//chunk_size
# n_chunks = 1


for i in range(0, n_chunks+1):
    with multiprocessing.Pool(processes=88) as pool_obj:
        pid_medications = pool_obj.map(get_medications_data, stays[i*chunk_size:(i+1)*chunk_size])

    for e in pid_medications[:]:
        temp = pd.DataFrame(e)
        final_meds = pd.concat([final_meds, temp], ignore_index=True)

    print("------------ Finished {} admissions ------------".format((i+1)*chunk_size))

    with open(PKL_PATH + 'pid_medications_icu.pkl', 'wb') as f:
        pickle.dump(final_meds, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'pid_medications_icu.pkl', 'rb') as f:
    pid_medications = pickle.load(f)
pid_medications.head()

### Multihot

The following cells are used to preproccess data into the multihot vector format:

In [ ]:
with open(PKL_PATH+'data_medications_icu.pkl', 'rb') as f:
    data_medications = pickle.load(f)

In [ ]:
data_medications.head()

In [ ]:
temp = data_medications[(data_medications.start_icu_day_id==0)][['subject_id',	'hadm_id',	'stay_id',	'itemid', 'label']].drop_duplicates()

In [ ]:
temp

In [ ]:
temp = pd.pivot_table(temp, index=['subject_id', 'hadm_id', 'stay_id'], columns='label')
temp = temp.droplevel(0, axis=1)

In [ ]:
temp

In [ ]:
new_names = ['med_' + col for col in temp.columns]
temp.columns = new_names
multihot_meds_icu_day_0 = temp.reset_index()

In [ ]:
multihot_meds_icu_day_0

Save

In [ ]:
with open(PKL_PATH + 'multihot_meds_icu_day_0.pkl', 'wb') as f:
    pickle.dump(multihot_meds_icu_day_0, f, protocol=pickle.HIGHEST_PROTOCOL)

# Chartevents Lab results

## Preparing the data

In [ ]:
pid_labevents.head()

In [ ]:
chartevents_labs['valuenum'] = chartevents_labs['valuenum'].fillna(-1).astype(float)

In [ ]:
chartevents_labs.head()

In [ ]:
chartevents_labs.info()

In [ ]:
chart_items_labs = chartevents_labs.itemid.unique()

for _, row in data_d_items[data_d_items.itemid.isin(chart_items_labs)].iterrows():
    print(row.itemid, '|', row.label, '|', row.abbreviation)

In [ ]:
data_labs = chartevents_labs.merge(data_icustays[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime']])

In [ ]:
data_labs['icu_day_id'] = data_labs['charttime'] - data_labs['intime']
data_labs['icu_day_id'] = [d.days for d in data_labs.loc[:, 'icu_day_id']]

In [ ]:
data_labs['icu_12h_window_id'] = data_labs['charttime'] - data_labs['intime']
data_labs['icu_12h_window_id'] = [(d.days*24 + d.seconds//3600)//12 for d in data_labs.loc[:, 'icu_12h_window_id']]

In [ ]:
data_labs = data_labs.sort_values(['subject_id', 'charttime', 'stay_id'])

In [ ]:
str_labs = data_labs[data_labs.value.str.contains('[a-zA-Z]')]
print(str_labs.abbreviation.unique())
str_labs

There are no str values so we proceed with making pid_dataset:

In [ ]:
chartevents_labs = data_labs[~(data_labs.value.str.contains('[a-zA-Z]'))]

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'chartevents_labs.pkl', 'wb') as f:
    pickle.dump(chartevents_labs, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'chartevents_labs.pkl', 'rb') as f:
    chartevents_labs = pickle.load(f)

In [ ]:
chartevents_labs.head(1)

## PID labs

### Test

In [ ]:
stays = data_icustays[data_icustays.los>2].stay_id

test = chartevents_labs[chartevents_labs.stay_id.isin(stays[:3])][['subject_id', 'hadm_id', 'stay_id', 'valuenum', 'valueuom','itemid', 'label', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id','itemid','label', 'icu_day_id'])\
        .agg({'valueuom':min, 'valuenum':'max'})\
            .reset_index()
test

In [ ]:
%%time
# print('1st row im progress..')
#  = temp.droplevel(0, axis=1)
# print('2st row im progress..')
test.columns = ['subject_id', 'hadm_id', 'stay_id', 'itemid', 'label', 'icu_day_id',  'valueuom', 'max_value']
# print('3st row im progress..')
test = test.sort_values(['subject_id', 'icu_day_id'])
print('4st row im progress..')
# test['labs_names'] = test['label']  + ' {' + test['max_value'].astype(str) + '}'
test['labs_codes'] = 'l' + test['itemid'].astype(str)  + ' {' + test['max_value'].astype(str) + '}'
print('5st row im progress..')
test_labs = test.groupby(['icu_day_id', 'subject_id', 'hadm_id', 'stay_id']).labs_codes.apply('; '.join).reset_index(name='labs_codes').sort_values(['subject_id', 'icu_day_id'])
test_labs

Implementation

In [ ]:
chartevents_labs.head(1)

In [ ]:
%%time
temp = chartevents_labs[['subject_id', 'hadm_id', 'stay_id', 'valuenum', 'valueuom','itemid', 'label', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id','itemid','label', 'icu_day_id'])\
        .agg({'valueuom':min, 'valuenum':'max'})\
            .reset_index()

In [ ]:
%%time
# print('1st row im progress..')
# temp = temp.droplevel(0, axis=1)
print('2st row im progress..')
temp.columns = ['subject_id', 'hadm_id', 'stay_id', 'itemid', 'label', 'icu_day_id',  'valueuom', 'max_value']
print('3st row im progress..')
temp = temp.sort_values(['subject_id', 'icu_day_id'])
print('4st row im progress..')
temp['labs_codes'] = 'l' + temp['itemid'].astype(str)  + ' {' + temp['max_value'].astype(str) + '}'
print('5st row im progress..')
pid_labs_codes = temp.groupby(['icu_day_id', 'subject_id', 'hadm_id', 'stay_id']).labs_codes.apply('; '.join).reset_index(name='labs_codes').sort_values(['subject_id', 'icu_day_id'])

In [ ]:
pid_labs_codes.head(1)

In [ ]:
pid_labs = pid_labs.merge(pid_labs_codes, on=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'], how='left')

In [ ]:
pid_labs

In [ ]:
pid_labs.loc[1, :].labs

In [ ]:
pid_labs.loc[1, :].labs_codes

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'pid_labs_icu.pkl', 'wb') as f:
    pickle.dump(pid_labs, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'pid_labs_icu.pkl', 'rb') as f:
    pid_labs = pickle.load(f)

# Demographics

## OMR

In [ ]:
data_omr.head()

In [ ]:
print(data_icustays.shape)
data_icustays.head()

In [ ]:
print('n of subject_ids which are NOT in data_omr table: ', data_icustays[~(data_icustays.subject_id.isin(data_omr.subject_id.unique()))].shape[0])

In [ ]:
data_demographics = data_icustays[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime']].merge(data_omr[['subject_id', 'chartdate', 'result_name', 'result_value']], on='subject_id', how='outer').drop_duplicates()
print(data_demographics.stay_id.unique().shape)
data_demographics.head(1)

In [ ]:
data_demographics = data_demographics[(data_demographics.chartdate > data_demographics.intime - pd.Timedelta(360, 'd'))&(data_demographics.chartdate < data_demographics.outtime)]\
                                    .sort_values(['subject_id', 'intime', 'chartdate'], ascending=False)\
                                        .drop_duplicates(['stay_id', 'result_name'], keep='first')\
                                            .sort_values(['subject_id', 'chartdate'])
print(data_demographics.stay_id.unique().shape)

In [ ]:
data_demographics['result_name'] = data_demographics['result_name'].str.replace("(Inches)|(kg/m2)|(Lbs)|(\(\))", '')

In [ ]:
data_demographics['name_result_value'] = data_demographics['result_name'] + ' {' + data_demographics['result_value'] + '} '

In [ ]:
data_demographics.head()

In [ ]:
data_demographics.result_name.unique()

In [ ]:
res = data_demographics.groupby(['hadm_id', 'stay_id']).name_result_value.apply(' '.join).reset_index(name='name_result_value')
res

## Age

In [ ]:
data_admissions.head(2)

In [ ]:
data_patients.head(2)

In [ ]:
data_demographics = pd.merge(data_admissions[['subject_id',	'hadm_id',	'admittime',	'dischtime', 'race']], \
                                data_patients[['subject_id',	'gender',	'anchor_age',	'anchor_year']], \
                                how='outer', on=['subject_id'])

In [ ]:
# get age of patients at the time of an admission
data_demographics['age_of_birth'] = data_demographics['anchor_year'] - data_demographics['anchor_age']
data_demographics['age'] = [value.year for value in data_demographics['admittime']] - data_demographics['age_of_birth']

# drop unnecessary columns
data_demographics = data_demographics.drop(['anchor_age','anchor_year', 'age_of_birth'], axis=1)

In [ ]:
data_demographics.age = data_demographics.age.fillna(-1)
data_demographics['age'] = data_demographics['age'].astype(int).astype(str)

In [ ]:
data_demographics.head()

## Combine 

In [ ]:
data_demographics = data_demographics.merge(res, on=['hadm_id'], how='outer')

In [ ]:
# build demographics sentence
data_demographics['demographics'] = data_demographics['race'].astype(str) + ' ' + data_demographics['gender'].astype(str) + ' {' +  data_demographics['age'].astype(str) + '} ' + data_demographics['name_result_value'].fillna('')

In [ ]:
data_demographics.head()

Afterwards changes:

In [ ]:
data_demographics.race.unique()

In [ ]:
data_demographics['race'] = data_demographics['race'].str.replace(r'-', '')
data_demographics['demographics'] = data_demographics['demographics'].str.replace(r'-', '')

In [ ]:
data_demographics[data_demographics.subject_id==subj]

Save:

In [ ]:
with open(PKL_PATH + 'pid_demographics_icu.pkl', 'wb') as f:
    pickle.dump(data_demographics, f, protocol=pickle.HIGHEST_PROTOCOL)

# Diagnoses

In [ ]:
with open('/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/pickles/' + 'hid_previous_diagnoses.pkl', 'rb') as f:
    hid_previous_diagnoses = pickle.load(f)

In [ ]:
data_prev_diagnoses = hid_previous_diagnoses.merge(data_icustays[['subject_id', 'hadm_id', 'stay_id']], on='hadm_id', how='right')
data_prev_diagnoses = data_prev_diagnoses[['subject_id', 'hadm_id', 'stay_id', 'previous_diags_icd', 'previous_diags_titles']]
print(data_prev_diagnoses.shape)
data_prev_diagnoses.head()

Save/load dataframe:

In [ ]:
with open(PKL_PATH + 'pid_prev_diagnoses_icu.pkl', 'wb') as f:
    pickle.dump(data_prev_diagnoses, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle5 as pickle 

with open(PKL_PATH + 'pid_prev_diagnoses_icu.pkl', 'rb') as f:
    pid_prev_diagnoses_icu = pickle.load(f)

In [ ]:
unique_duags = pid_prev_diagnoses_icu.previous_diags_icd.str.split(' ').explode().unique()
len(unique_duags)

In [ ]:
data_diagnoses.icd_code.unique().shape

# Compiling the dataset

## Loading dataframes

In [ ]:
LABELS_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/pickles_2/aki_stage_labels.pkl'

with open(LABELS_PATH, 'rb') as f:
    aki_stage_labels = pickle.load(f)

In [ ]:
with open(PKL_PATH + 'aki_status_icu.pkl', 'rb') as f:
    aki_status_icu = pickle.load(f)
aki_status_icu = aki_status_icu.groupby(['subject_id',	'hadm_id',	'stay_id', 'icu_day_id', 'specimen_id']).sum().reset_index()
aki_status_icu.head(5)

In [ ]:
stay = 30696680
aki_status_icu[aki_status_icu.stay_id==stay]

In [ ]:
stay = 38020798
subj = 12380418
aki_status_icu[aki_status_icu.stay_id==stay]

In [ ]:
with open(PKL_PATH + 'pid_vitals_icu.pkl', 'rb') as f:
    pid_vitals = pickle.load(f)
print('unique stays: ', pid_vitals.stay_id.unique().shape[0])
pid_vitals.head()

In [ ]:
pid_vitals[pid_vitals.stay_id==stay]

In [ ]:
chartevents_vitals[chartevents_vitals.stay_id==stay].sort_values('charttime')

In [ ]:
with open(PKL_PATH + 'pid_outputs.pkl', 'rb') as f:
    pid_outputs = pickle.load(f)
print('unique stays: ', pid_outputs.stay_id.unique().shape[0])
pid_outputs[pid_outputs.subject_id==10001884].head()

In [ ]:
pid_outputs[pid_outputs.stay_id==stay]

In [ ]:
data_outputevents[data_outputevents.stay_id==stay].sort_values('charttime')

In [ ]:
with open(PKL_PATH + 'pid_medications_icu.pkl', 'rb') as f:
    pid_medications = pickle.load(f)
pid_medications.rename(columns={'_12h_window_id':'icu_12h_window_id'}, inplace=True)
print('unique stays: ', pid_medications.stay_id.unique().shape[0])
pid_medications[pid_medications.subject_id==10001884].head()

In [ ]:
pid_medications[pid_medications.stay_id==stay]

In [ ]:
data_icustays[data_icustays.stay_id==stay]

In [ ]:
with open(PKL_PATH + 'pid_labs_icu.pkl', 'rb') as f:
    pid_labs = pickle.load(f)
print('unique stays: ', pid_labs.stay_id.unique().shape[0])
pid_labs.head()

In [ ]:
with open(PKL_PATH + 'pid_demographics_icu.pkl', 'rb') as f:
    pid_demographics_icu =  pickle.load(f)
print('unique stays: ', pid_demographics_icu.stay_id.unique().shape[0])
pid_demographics_icu.head()

In [ ]:
pid_demographics_icu[pid_demographics_icu.subject_id==16093826]

In [ ]:
data_patients[data_patients.subject_id==16093826]

In [ ]:
with open(PKL_PATH + 'pid_prev_diagnoses_icu.pkl', 'rb') as f:
    pid_prev_diagnoses_icu =  pickle.load(f)
print('unique stays: ', pid_prev_diagnoses_icu.stay_id.unique().shape[0])
pid_prev_diagnoses_icu.head()

In [ ]:
pid_prev_diagnoses_icu[pid_prev_diagnoses_icu.subject_id==subj]

In [ ]:
data_admissions[data_admissions.subject_id==subj]

## Compiling dataframes

We first concatenate vitals, outputs and medication since they are 12h-level:

In [ ]:
# continuous info
pid_dataset_icu_cont = pid_vitals.merge(pid_outputs, how='outer', on=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id', 'icu_12h_window_id'])\
                            .merge(pid_medications, how='outer', on=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id', 'icu_12h_window_id'])\
                                .merge(pid_labs, how='outer', on=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'])
                            

In [ ]:
pid_dataset_icu_cont.head(1)

In [ ]:
# static info
pid_dataset_icu_static = pid_prev_diagnoses_icu.merge(pid_demographics_icu[['subject_id',	'hadm_id',	'stay_id',	'demographics']], how='outer', on=['subject_id',	'hadm_id'])
pid_dataset_icu_static = pid_dataset_icu_static.rename(columns={'stay_id_x':'stay_id'}).drop(columns=['stay_id_y']).drop_duplicates()                       
                    

In [ ]:
pid_dataset_icu_static.head(1)

In [ ]:
# combine two dataframes
pid_dataset_icu = pid_dataset_icu_cont.merge(pid_dataset_icu_static, on=['subject_id',	'hadm_id',	'stay_id'])

In [ ]:
# add labels
# pid_dataset_icu = pid_dataset_icu.merge(aki_status_icu[['subject_id',	'hadm_id',	'stay_id',	'day_id',	'icu_day_id',	'icu_12h_window_id', 'AKI_1', 'AKI_2', 'AKI_3']], how='outer')
pid_dataset_icu = pid_dataset_icu.merge(aki_stage_labels[['subject_id',	'hadm_id',	'stay_id',	'day_id',	'icu_day_id', 'AKI_1', 'AKI_2', 'AKI_3', 'NO_AKI', 'ANY_AKI']], how='outer')

In [ ]:
# filter out nans
pid_dataset_icu = pid_dataset_icu[~((pid_dataset_icu.outputs.isna())&(pid_dataset_icu.medications_names.isna())& (pid_dataset_icu.vitals.isna()))]

In [ ]:
pid_dataset_icu.columns = ['icu_12h_window_id','icu_day_id', 'subject_id', 'hadm_id', 'stay_id','vitals_names', 'vitals_codes','outputs_names', 'outputs_codes',\
     'medications_names', 'medications_codes','labs_names', 'labs_codes', 'previous_diags_codes','previous_diags_names','demographics',  'day_id','AKI_1', 'AKI_2', 'AKI_3', 'NO_AKI', 'ANY_AKI']
         
# rearrange columns
pid_dataset_icu = pid_dataset_icu[['subject_id',	'hadm_id',	'stay_id', 'day_id', 'icu_12h_window_id',	'icu_day_id', 'demographics',	'previous_diags_codes',	\
    'previous_diags_names', 'vitals_names','vitals_codes',	'labs_names','labs_codes',	'outputs_names','outputs_codes',	'medications_names','medications_codes', \
        'AKI_1', 'AKI_2', 'AKI_3', 'NO_AKI', 'ANY_AKI']]

In [ ]:
pid_dataset_icu['demographics'] = pid_dataset_icu['demographics'].fillna('')
pid_dataset_icu['previous_diags_codes'] = pid_dataset_icu['previous_diags_codes'].fillna('')
pid_dataset_icu['previous_diags_names'] = pid_dataset_icu['previous_diags_names'].fillna('')
pid_dataset_icu['vitals_names'] = pid_dataset_icu['vitals_names'].fillna('')
pid_dataset_icu['vitals_codes'] = pid_dataset_icu['vitals_codes'].fillna('')
pid_dataset_icu['labs_names'] = pid_dataset_icu['labs_names'].fillna('')
pid_dataset_icu['labs_codes'] = pid_dataset_icu['labs_codes'].fillna('')
pid_dataset_icu['outputs_names'] = pid_dataset_icu['outputs_names'].fillna('')
pid_dataset_icu['outputs_codes'] = pid_dataset_icu['outputs_codes'].fillna('')
pid_dataset_icu['medications_names'] = pid_dataset_icu['medications_names'].fillna('')
pid_dataset_icu['medications_codes'] = pid_dataset_icu['medications_codes'].fillna('')

pid_dataset_icu['AKI_1'] = pid_dataset_icu['AKI_1'].fillna(0)
pid_dataset_icu['AKI_2'] = pid_dataset_icu['AKI_2'].fillna(0)
pid_dataset_icu['AKI_3'] = pid_dataset_icu['AKI_3'].fillna(0)
pid_dataset_icu['NO_AKI'] = pid_dataset_icu['NO_AKI'].fillna(0)
pid_dataset_icu['ANY_AKI'] = pid_dataset_icu['ANY_AKI'].fillna(0)

In [ ]:
# add concatenated info
pid_dataset_icu['icu_12h_info_codes'] =  pid_dataset_icu['vitals_codes'] + ' $ ' +  pid_dataset_icu['outputs_codes'] + ' $ ' + pid_dataset_icu['medications_codes']
pid_dataset_icu['icu_12h_info_names'] =   pid_dataset_icu['vitals_names'] + ' $ ' + pid_dataset_icu['outputs_names'] + ' $ ' + pid_dataset_icu['medications_names']

In [ ]:
pid_dataset_icu.isna().sum()

In [ ]:
# filter stays with no info for 0,1 days in icu
stays_12h_0123 = []
stays_without_12h_0123 = []
for stay in pid_dataset_icu.stay_id.unique():
    if np.isin(pid_dataset_icu[pid_dataset_icu.stay_id==stay].icu_12h_window_id.values, np.array([0,1,2,3])).sum() >3:
        stays_12h_0123.append(stay)
    elif np.isin(pid_dataset_icu[pid_dataset_icu.stay_id==stay].icu_day_id.values, np.array([0,1])).sum()>1:
        stays_12h_0123.append(stay)
        
    else:
        stays_without_12h_0123.append(stay)

In [ ]:
print(len(stays_12h_0123))
print(len(stays_without_12h_0123))

In [ ]:
print(len(stays_12h_0123))
print(len(stays_without_12h_0123))

# Tokenizer ICU

In [ ]:
adm = 28994087
pid_dataset_icu[pid_dataset_icu.hadm_id==adm]

In [ ]:
pid_dataset_icu.columns

In [ ]:
# patient - level info: demographics, last body measurements
pid_dataset_icu.iloc[0, 6]

In [ ]:
# 12h info: vitals, medications, outputs
pid_dataset_icu.iloc[0, -2]

In [ ]:
# 24h info: labs
pid_dataset_icu.iloc[0, -9]

In [ ]:
# 24 day info codes
pid_dataset_icu.iloc[0, -2]

In [ ]:
# 24 day info names
pid_dataset_icu.iloc[0, -1]

In [ ]:
pid_dataset_icu.stay_id.unique().shape[0]

In [ ]:
pid_dataset_icu.columns

Save splits:

In [ ]:
print(len(train_stays))
print(len(test_stays))
print(len(val_stays))

In [ ]:
pid_train_dataset_icu = pid_dataset_icu[pid_dataset_icu.stay_id.isin(train_stays)]
pid_test_dataset_icu = pid_dataset_icu[pid_dataset_icu.stay_id.isin(test_stays)]
pid_val_dataset_icu = pid_dataset_icu[pid_dataset_icu.stay_id.isin(val_stays)]

In [ ]:
with open(DF_PATH + 'pid_train_dataset_icu.pkl', 'wb') as f:
    pickle.dump(pid_train_dataset_icu, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(DF_PATH + 'pid_test_dataset_icu.pkl', 'wb') as f:
    pickle.dump(pid_test_dataset_icu, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(DF_PATH + 'pid_val_dataset_icu.pkl', 'wb') as f:
    pickle.dump(pid_val_dataset_icu, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(PKL_PATH + 'aki_stage_labels_second_day.pkl', 'rb') as f:
    aki_stage_labels_second_day = pickle.load(f)
aki_stage_labels_second_day

# Split dataset by stay_id and save

In [ ]:
def difference(lst1, lst2):
    lst3 = [value for value in lst1 if value not in lst2]
    #return lst3
    return lst3

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    #return lst3
    return lst3

In [ ]:
with open(DF_PATH + 'pid_train_dataset_icu.pkl', 'rb') as f:
    pid_train_dataset_icu = pickle.load(f)

with open(DF_PATH + 'pid_test_dataset_icu.pkl', 'rb') as f:
    pid_test_dataset_icu = pickle.load(f)

with open(DF_PATH + 'pid_val_dataset_icu.pkl', 'rb') as f:
    pid_val_dataset_icu = pickle.load(f)

LABELS_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/pickles_2/aki_stage_labels.pkl'

with open(LABELS_PATH, 'rb') as f:
    aki_stage_labels = pickle.load(f)

In [ ]:
dataset = pd.concat([pid_train_dataset_icu, pid_test_dataset_icu, pid_val_dataset_icu])
stays = dataset.stay_id.unique()
len(stays)

In [ ]:
aki_stage_labels = aki_stage_labels[aki_stage_labels.stay_id.isin(stays)&(aki_stage_labels.icu_day_id==1)]
aki_stage_labels.stay_id.unique().shape

In [ ]:
pid_train_dataset_icu = pid_train_dataset_icu[pid_train_dataset_icu.stay_id.isin(aki_stage_labels.stay_id.unique())]
pid_test_dataset_icu = pid_test_dataset_icu[pid_test_dataset_icu.stay_id.isin(aki_stage_labels.stay_id.unique())]
pid_val_dataset_icu = pid_val_dataset_icu[pid_val_dataset_icu.stay_id.isin(aki_stage_labels.stay_id.unique())]

In [ ]:
print(pid_train_dataset_icu.stay_id.unique().shape[0])
print(pid_test_dataset_icu.stay_id.unique().shape[0])
print(pid_val_dataset_icu.stay_id.unique().shape[0])

In [ ]:
pid_train_dataset_icu = pid_train_dataset_icu.replace(r'[{}]', '', regex=True)
pid_test_dataset_icu = pid_test_dataset_icu.replace(r'[{}]', '', regex=True)
pid_val_dataset_icu = pid_val_dataset_icu.replace(r'[{}]', '', regex=True)

In [ ]:
for i,x in pid_test_dataset_icu[:50].groupby(['subject_id', 'hadm_id', 'stay_id']):
    print(i[2])

In [ ]:
for i, x in pid_test_dataset_icu.groupby(['subject_id', 'hadm_id', 'stay_id']):
    if not os.path.exists(DF_PATH+'test_data'):
        os.mkdir(DF_PATH+'test_data')
    p = os.path.join(DF_PATH+'test_data', "icu_stay_{}.csv".format(str(i[2]).lower()))
    x.to_csv(p, index=False)
    print(f'Saved to {p}')

In [ ]:
pd.read_csv('/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/dataframes_2/test_data/icu_stay_34868320.csv')

In [ ]:
for i, x in pid_train_dataset_icu.groupby(['subject_id', 'hadm_id', 'stay_id']):
    if not os.path.exists(DF_PATH+'train_data'):
        os.mkdir(DF_PATH+'train_data')
    p = os.path.join(DF_PATH+'train_data', "icu_stay_{}.csv".format(str(i[2]).lower()))
    x.to_csv(p, index=False)
    print(f'Saved to {p}')

In [ ]:
for i, x in pid_val_dataset_icu.groupby(['subject_id', 'hadm_id', 'stay_id']):
    if not os.path.exists(DF_PATH+'val_data'):
        os.mkdir(DF_PATH+'val_data')
    p = os.path.join(DF_PATH+'val_data', "icu_stay_{}.csv".format(str(i[2]).lower()))
    x.to_csv(p, index=False)
    print(f'Saved to {p}')

### Tokenizer txt files:

In [ ]:
pid_train_dataset_icu.stay_id.unique().shape[0]

In [ ]:
tokenizer_pid_dataset_icu = pd.concat([pid_train_dataset_icu, pid_val_dataset_icu]).fillna('')
print(tokenizer_pid_dataset_icu.stay_id.unique().shape[0])
print(tokenizer_pid_dataset_icu.shape)

In [ ]:
pid_train_dataset_icu.iloc[:1, :10]

In [ ]:
pid_train_dataset_icu.iloc[:1, 10:]

In [ ]:
from tqdm.auto import tqdm

def create_text_seq(data, txt_dir, titles=False):
    if titles:
        previous_diags = 'previous_diags_titles'
        info = 'icu_12h_info_titles'
        labs = 'labs'
    else:
        previous_diags = 'previous_diags_icd'
        info = 'icu_12h_info' 
        labs = 'labs_codes'

    text_data = []
    file_count = 0
    hadm_id_d = {}

    for _, row in tqdm(data.iterrows()):
        hadm_id = row['hadm_id']
        icu_day_id = row['icu_day_id']
        if hadm_id not in hadm_id_d:
            hadm_id_d[hadm_id] = []
            text_data.append(row['demographics'])
            text_data.append(row[previous_diags])
        if icu_day_id not in hadm_id_d[hadm_id]:
            hadm_id_d[hadm_id].append(icu_day_id)
            text_data.append(row[labs])
        text_data.append(row[info])
        if (len(text_data) % 5000) == 0:
            # once we hit the 5K mark, save to file
            if not os.path.exists(txt_dir):
                os.makedirs(txt_dir)
            with open(f'{txt_dir}/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
                fp.write('\n'.join(text_data))
            text_data = []
            file_count += 1
            print(f"Saved {5000 + (5000 * (file_count - 1))} samples")

    if (len(data) % 5000) != 0:
        with open(f'{txt_dir}/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        print(f"Saved {5000 + (5000 * (file_count - 1)) + len(text_data)} samples")

In [ ]:
TXT_DIR_TRAIN = '/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/txt_files/icu_train'
create_text_seq(data=tokenizer_pid_dataset_icu, txt_dir=TXT_DIR_TRAIN, titles=False)

In [ ]:
TXT_DIR_TRAIN = '/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/txt_files/icu_train_titles'
create_text_seq(data=tokenizer_pid_dataset_icu, txt_dir=TXT_DIR_TRAIN, titles=True)

# Checking

In [ ]:
aki_status_icu[aki_status_icu.stay_id==31090461]

In [ ]:
# right on
pid_dataset_icu[pid_dataset_icu.stay_id==31090461]

In [ ]:
# left on
pid_dataset_icu[pid_dataset_icu.stay_id==31090461]

In [ ]:
data_admissions[data_admissions.subject_id==10840421].sort_values('admittime')

In [ ]:
data_icustays[data_icustays.subject_id==10840421]

# XGBoost preprocessing

In [ ]:
XGB_DATA_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/xgb/dataframes/'

## Demographics

In [ ]:
with open(PKL_PATH + 'pid_demographics_icu.pkl', 'rb') as f:
    pid_demographics_icu =  pickle.load(f)
print('unique stays: ', pid_demographics_icu.stay_id.unique().shape[0])
pid_demographics_icu.head()

In [ ]:
data_icustays[data_icustays.subject_id==10004457]

In [ ]:
pid_demographics_icu = pid_demographics_icu.merge(data_icustays, on=['subject_id', 'hadm_id'], how='outer')
# pid_demographics_icu[pid_demographics_icu.admittime>pid_demographics_icu.intime]

In [ ]:
data_omr.head()

In [ ]:
data_demographics = data_icustays[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime']].merge(data_omr[['subject_id', 'chartdate', 'result_name', 'result_value']], on='subject_id', how='inner').drop_duplicates()
print(data_demographics.stay_id.unique().shape)
data_demographics.head(1)

In [ ]:
data_demographics[data_demographics.subject_id==p]

In [ ]:
data_omr[data_omr.subject_id==p]

In [ ]:
data_demographics = data_demographics[(data_demographics.chartdate > data_demographics.intime - pd.Timedelta(360, 'd'))&(data_demographics.chartdate < data_demographics.outtime)]\
                                    .sort_values(['subject_id', 'intime', 'chartdate'], ascending=False)\
                                        .drop_duplicates(['stay_id', 'result_name'], keep='first')\
                                            .sort_values(['subject_id', 'chartdate'])
print(data_demographics.stay_id.unique().shape)

In [ ]:
data_demographics[data_demographics.subject_id==p]

In [ ]:
data_demographics.head(1)

Get omr numeric data

In [ ]:
blood_pressure_df = data_demographics[data_demographics.result_name.str.lower().str.contains('pressure')]
data_demographics_omr = data_demographics[~(data_demographics.result_name.str.lower().str.contains('pressure'))]

In [ ]:
data_demographics_omr['result_value'] = pd.to_numeric(data_demographics_omr['result_value'], errors='coerce')

In [ ]:
data_demographics_omr = pd.pivot_table(data_demographics_omr, values='result_value', index=['subject_id', 'hadm_id', 'stay_id'], columns='result_name').reset_index()

In [ ]:
data_demographics_omr.head(1)

In [ ]:
data_demographics_omr = data_demographics_omr[['subject_id',	'hadm_id',	'stay_id', 'BMI (kg/m2)',	'Height (Inches)',	'Weight (Lbs)']]
data_demographics_omr.columns = ['subject_id',	'hadm_id',	'stay_id', 'BMI',	'Height',	'Weight']

In [ ]:
data_demographics_omr.head(1)

In [ ]:
data_demographics_omr.isna().sum()

In [ ]:
p = 15926317
data_demographics_omr[data_demographics_omr.subject_id==p]

Get BP data

In [ ]:
blood_pressure_df.drop_duplicates('result_name')

In [ ]:
systolic_l, diastolic_l = [], []
for pair in blood_pressure_df.result_value.str.split('/').values:
    systolic_l.append(pair[0])
    diastolic_l.append(pair[1])

In [ ]:
blood_pressure_df['systolic'] = systolic_l
blood_pressure_df['diastolic'] = diastolic_l

In [ ]:
blood_pressure_df.head(2)

In [ ]:
bp_df = pd.pivot_table(blood_pressure_df, values=['systolic', 'diastolic'], index=['subject_id', 'hadm_id', 'stay_id'], columns='result_name')\
            .droplevel(0, axis=1).reset_index()

In [ ]:
bp_df.columns = ['subject_id',	'hadm_id',	'stay_id', 'diastolic_bp', 'diastolic_bp_lying', 'diastolic_bp_sitting', 'diastolic_bp_standing', 'diastolic_bp_standing_1m', 'diastolic_bp_standing_3m', 'systolic_bp', 'systolic_bp_lying', 'systolic_bp_sitting', 'systolic_bp_standing', 'systolic_bp_standing_1m', 'systolic_bp_standing_3m' ]

In [ ]:
bp_df = bp_df[['subject_id',	'hadm_id',	'stay_id', 'diastolic_bp', 'systolic_bp']]

In [ ]:
bp_df.head()

In [ ]:
bp_df.isna().sum()

Combine two frames:


In [ ]:
data_demographics_omr = data_demographics_omr.merge(bp_df, how='outer')

In [ ]:
data_demographics_omr.head(1)

In [ ]:
data_demographics_omr.isna().sum()

In [ ]:
p = 15926317
pid_demographics_icu[pid_demographics_icu.subject_id==p]

In [ ]:
data_demographics_omr[data_demographics_omr.subject_id==p]

In [ ]:
with open(XGB_DATA_PATH + 'data_demographics_omr.pkl', 'wb') as f:
    pickle.dump(data_demographics_omr, f, protocol=pickle.HIGHEST_PROTOCOL)

Combine with Race, gender, ethnicity

In [ ]:
pid_demographics_icu[pid_demographics_icu.subject_id==p]

In [ ]:
# check how many stay_id_x and y are not the same
print(pid_demographics_icu[~(pid_demographics_icu.stay_id_x==pid_demographics_icu.stay_id_y)].shape)
pid_demographics_icu[~(pid_demographics_icu.stay_id_x==pid_demographics_icu.stay_id_y)].isna().sum().sort_values()

In [ ]:
pid_demographics_icu['stay_id_x'] = pid_demographics_icu['stay_id_x'].fillna(pid_demographics_icu['stay_id_y']).fillna(0)

In [ ]:
pid_demographics_icu = pid_demographics_icu.rename(columns={'stay_id_x':'stay_id'})

In [ ]:
# demo = pid_demographics_icu[~(pid_demographics_icu.stay_id.isna())][['subject_id',	'hadm_id',	'stay_id', 'race',	'gender', 'age']]
demo = pid_demographics_icu[['subject_id',	'hadm_id',	'stay_id', 'race',	'gender', 'age']]
# data_demographics = demo.merge(data_demographics_omr, on=['subject_id',	'hadm_id',	'stay_id'], how='outer' )
# data_demographics = demo.merge(data_demographics_omr, on=['subject_id',	'hadm_id'], how='outer' )

In [ ]:
p = 15926317
pid_demographics_icu[data_demographics.subject_id==p]

In [ ]:
data_icustays[data_icustays.subject_id==p]

In [ ]:
# data_demographics['stay_id_x'] = data_demographics['stay_id_x'].fillna(data_demographics['stay_id_y']).fillna(0)
# data_demographics = data_demographics.rename(columns={'stay_id_x':'stay_id'})

In [ ]:
print(data_demographics.shape)
data_demographics.head()

In [ ]:
data_demographics.isna().sum()

Cast categorical to numbers:

In [ ]:
pid_demographics_icu.gender[pid_demographics_icu.gender == 'F'] = 1
pid_demographics_icu.gender[pid_demographics_icu.gender == 'M'] = 2

In [ ]:
pd.factorize(pid_demographics_icu['race'])

In [ ]:
pid_demographics_icu['race'] = pd.factorize(pid_demographics_icu['race'])[0] + 1

In [ ]:
pid_demographics_icu['stay_id'] = pid_demographics_icu['stay_id'].fillna(0).astype('int64')
pid_demographics_icu['hadm_id'] = pid_demographics_icu['hadm_id'].fillna(0).astype('int64')

In [ ]:
# pid_demographics_icu = pid_demographics_icu.drop('stay_id_y', axis=1)

In [ ]:
pid_demographics_icu.head()

In [ ]:
data_icustays[data_icustays.subject_id==p]

In [ ]:
data_omr[data_omr.subject_id==p]

Save

In [ ]:
# with open(XGB_DATA_PATH + 'data_demographics_xgb.pkl', 'wb') as f:
#     pickle.dump(data_demographics, f, protocol=pickle.HIGHEST_PROTOCOL)

## Lab results

In [ ]:
with open(PKL_PATH + 'chartevents_labs.pkl', 'rb') as f:
    chartevents_labs = pickle.load(f)
chartevents_labs.head()

### Test

In [ ]:
stays = data_icustays[data_icustays.los>2].stay_id

test = chartevents_labs[chartevents_labs.stay_id.isin(stays[:3])][['subject_id', 'hadm_id', 'stay_id', 'valuenum', 'valueuom','itemid', 'label', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id'])\
        .agg({ 'valuenum':['min','max']})\
            .reset_index()
test

In [ ]:
test = test.droplevel(0, axis=1)
test.columns = ['subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id',  'min_value', 'max_value']
test = test.sort_values(['subject_id', 'icu_day_id'])
test

In [ ]:
# test = pd.pivot_table(test, index=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'], columns='label', values=['min_value', 'max_value'])
# test = test.droplevel(0, axis=1)
new_names = [*['min_'+col for col in test.iloc[:, :63].columns], *['max_'+col for col in test.iloc[:, 63:].columns]]
test.columns = new_names
test = test.reset_index()

In [ ]:
test.head(1)

### Implementation:

In [ ]:
temp = chartevents_labs[['subject_id', 'hadm_id', 'stay_id', 'valuenum', 'valueuom','itemid', 'label', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id'])\
        .agg({ 'valuenum':['min','max']})\
            .reset_index()
temp

In [ ]:
temp = temp.droplevel(0, axis=1)
temp.columns = ['subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id',  'min_value', 'max_value']
temp = temp.sort_values(['subject_id', 'icu_day_id'])
temp

In [ ]:
temp = pd.pivot_table(temp, index=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'], columns='label', values=['min_value', 'max_value'])
temp = temp.droplevel(0, axis=1)

In [ ]:
for i, col in enumerate(temp.columns):
    print(i, col)

In [ ]:
new_names = [*['min_'+col for col in temp.iloc[:, :108].columns], *['max_'+col for col in temp.iloc[:, 108:].columns]]
temp.columns = new_names
temp = temp.reset_index()

In [ ]:
data_labs_xgb = temp.copy()

Save

In [ ]:
# with open(XGB_DATA_PATH + 'data_labs_xgb.pkl', 'wb') as f:
#     pickle.dump(data_labs_xgb, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(XGB_DATA_PATH + 'data_labs_xgb.pkl', 'rb') as f:
    data_labs_xgb = pickle.load(f)
data_labs_xgb.head()

### Multihot

Test

In [ ]:
stays = data_icustays[data_icustays.los>2].stay_id

test = chartevents_labs[(chartevents_labs.stay_id.isin(stays[:3]))&(chartevents_labs.icu_day_id==0)][['subject_id', 'hadm_id', 'stay_id', 'charttime', 'valuenum', 'valueuom','itemid', 'label', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id'])\
        .agg({'valuenum': [lambda x: x.iloc[0], lambda x: x.iloc[-1]]})\
            .reset_index()
test

In [ ]:
test = test.droplevel(0, axis=1)
test.columns = ['subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id',  'first_value', 'last_value']
test = test.sort_values(['subject_id', 'icu_day_id'])
test

In [ ]:
test = pd.pivot_table(test, index=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'], columns='label', values=['first_value', 'last_value'])
test = test.droplevel(0, axis=1)
test

In [ ]:
for i, col in enumerate(test.columns):
    print(i, col)

In [ ]:
new_names = [*['firsl_'+col for col in test.iloc[:, :58].columns], *['last_' + col for col in test.iloc[:, 58:].columns]]
test.columns = new_names
test = test.reset_index()

In [ ]:
test

In [ ]:
chartevents_labs[(chartevents_labs.stay_id==37510196)&(chartevents_labs.label=='ALT')]

Implementation

In [ ]:
temp = chartevents_labs[(chartevents_labs.icu_day_id==0)][['subject_id', 'hadm_id', 'stay_id', 'charttime', 'valuenum', 'valueuom','itemid', 'label', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id'])\
        .agg({'valuenum': [lambda x: x.iloc[0], lambda x: x.iloc[-1]]})\
            .reset_index()

In [ ]:
temp

In [ ]:
temp = temp.droplevel(0, axis=1)
temp.columns = ['subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id',  'first_value', 'last_value']
temp = temp.sort_values(['subject_id', 'icu_day_id'])
temp

In [ ]:
temp = pd.pivot_table(temp, index=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'], columns='label', values=['first_value', 'last_value'])
temp = temp.droplevel(0, axis=1)

In [ ]:
temp

In [ ]:
for i, col in enumerate(temp.columns):
    print(i, col)

In [ ]:
new_names = [*['first_'+col for col in temp.iloc[:, :104].columns], *['last_' + col for col in temp.iloc[:, 104:].columns]]
temp.columns = new_names
multihot_labs_day_0 = temp.reset_index()

In [ ]:
for k,v in  multihot_labs_day_0.isna().sum().sort_values(ascending=False).iteritems():
    print(k, v)

In [ ]:
print(len(multihot_labs_day_0))
print(multihot_labs_day_0.hadm_id.unique().shape[0])

In [ ]:
multihot_labs_day_0[multihot_labs_day_0.duplicated('hadm_id')].sort_values(['hadm_id', 'icu_day_id'])

Save

In [ ]:
with open(PKL_PATH + 'multihot_labs_icu_day_0.pkl', 'wb') as f:
    pickle.dump(multihot_labs_day_0, f, protocol=pickle.HIGHEST_PROTOCOL)

## Vitals

In [ ]:
with open(PKL_PATH + 'chartevents_vitals.pkl', 'rb') as f:
    chartevents_vitals = pickle.load(f)

In [ ]:
num_chartevents_vitals = chartevents_vitals[~(chartevents_vitals.value.str.contains('[a-zA-Z]'))]
num_chartevents_vitals['valuenum'] = num_chartevents_vitals['valuenum'].astype(float)

In [ ]:
# tried to use label OR abbrevation OR itemid
temp = num_chartevents_vitals[['subject_id', 'hadm_id', 'stay_id', 'itemid', 'valuenum', 'valueuom', 'abbreviation', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id', 'abbreviation', 'icu_day_id'])\
        .agg({'valuenum':['min', 'max']})\
            .reset_index()

In [ ]:
temp = temp.droplevel(0, axis=1)
temp.columns = ['subject_id', 'hadm_id', 'stay_id', 'label', 'icu_day_id',  'min_value', 'max_value']
temp = temp.sort_values(['subject_id', 'icu_day_id'])
temp

In [ ]:
temp = pd.pivot_table(temp, index=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'], columns='label', values=['min_value', 'max_value'])
temp = temp.droplevel(0, axis=1)

In [ ]:
for i, col in enumerate(temp.columns):
    print(i, col)

In [ ]:
new_names = [*['min_'+col for col in temp.iloc[:, :33].columns], *['max_'+col for col in temp.iloc[:, 33:].columns]]
temp.columns = new_names
temp = temp.reset_index()

In [ ]:
data_vitals_xgb = temp.copy()

In [ ]:
data_vitals_xgb.head()

Save

In [ ]:
with open(XGB_DATA_PATH + 'data_vitals_xgb.pkl', 'wb') as f:
    pickle.dump(data_vitals_xgb, f, protocol=pickle.HIGHEST_PROTOCOL)

### Multihot

In [ ]:
chartevents_vitals.head(1)

In [ ]:
grouped_vitals = chartevents_vitals.groupby(['subject_id', 'hadm_id', 'stay_id', 'abbreviation', 'icu_day_id']).count()


The minimum value of vitals is 1 and maximum is 266. 75% of the samples have around 24 measurements.

In [ ]:
grouped_vitals.sort_values('value').describe()

In [ ]:
temp = num_chartevents_vitals[num_chartevents_vitals.icu_day_id==0][['subject_id', 'hadm_id', 'stay_id', 'itemid', 'valuenum', 'valueuom', 'abbreviation', 'icu_day_id', 'icu_12h_window_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id', 'abbreviation', 'icu_day_id'])\
        .agg({'valuenum':[lambda x: x.iloc[0], lambda x: x.iloc[-1], 'median', 'min', 'max']})\
            .reset_index()

In [ ]:
temp

In [ ]:
chartevents_vitals[(chartevents_vitals.stay_id==37510196)&(chartevents_vitals.icu_day_id==0)&(chartevents_vitals.abbreviation=='EtCO2')]

In [ ]:
temp = temp.droplevel(0, axis=1)
temp.columns = ['subject_id', 'hadm_id', 'stay_id', 'abbreviation', 'icu_day_id',  'first_value', 'last_value', 'median_value',	'min_value', 'max_value']
temp = temp.sort_values(['subject_id', 'icu_day_id'])
temp

In [ ]:
temp = pd.pivot_table(temp, index=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'], columns='abbreviation', values=['first_value', 'last_value', 'median_value',	'min_value', 'max_value'])
temp = temp.droplevel(0, axis=1)

In [ ]:
temp

In [ ]:
for i, col in enumerate(temp.columns):
    print(i, col)

In [ ]:
new_names = [*['first_'+col for col in temp.iloc[:, :33].columns], *['last_' + col for col in temp.iloc[:, 33:66].columns],\
     *['median_' + col for col in temp.iloc[:, 66:99].columns], *['min_' + col for col in temp.iloc[:, 99:132].columns], \
        *['max_' + col for col in temp.iloc[:, 132:].columns]]
temp.columns = new_names
multihot_vitals_day_0 = temp.reset_index()

In [ ]:
print(len(multihot_vitals_day_0))
print(multihot_vitals_day_0.hadm_id.unique().shape[0])

In [ ]:
multihot_vitals_day_0.isna().sum().sort_values(ascending=False)

Save

In [ ]:
with open(PKL_PATH + 'multihot_vitals_icu_day_0.pkl', 'wb') as f:
    pickle.dump(multihot_vitals_day_0, f, protocol=pickle.HIGHEST_PROTOCOL)

## Outputs

In [ ]:
with open(PKL_PATH + 'data_outputevents.pkl', 'rb') as f:
    data_outputevents = pickle.load(f)

In [ ]:
%%time
temp = data_outputevents[['subject_id', 'hadm_id', 'stay_id', 'value', 'abbreviation', 'icu_day_id']]\
    .groupby(['subject_id', 'hadm_id', 'stay_id', 'abbreviation', 'icu_day_id'])\
        .agg({ 'value':sum})\
            .reset_index().sort_values(['subject_id', 'icu_day_id'])
temp.head(1)

In [ ]:
temp = pd.pivot_table(temp, index=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id'], columns='abbreviation', values='value').reset_index()
temp['output'] = temp['Foley'].fillna(0) + temp['Void'].fillna(0) 

In [ ]:
data_output_xgb = temp.copy()

In [ ]:
data_output_xgb = data_output_xgb[['subject_id', 'hadm_id', 'stay_id', 'icu_day_id', 'output']]

In [ ]:
data_output_xgb = data_output_xgb.reset_index(drop=True)

In [ ]:
data_output_xgb.head()

Save

In [ ]:
with open(XGB_DATA_PATH + 'data_output_xgb.pkl', 'wb') as f:
    pickle.dump(data_output_xgb, f, protocol=pickle.HIGHEST_PROTOCOL)

## Diagnoses

In [ ]:
cirrhosis_icd = data_d_icd_diagnoses[data_d_icd_diagnoses.long_title.str.contains('cirrhosis')].icd_code.to_list()
cad_icd = data_d_icd_diagnoses[data_d_icd_diagnoses.long_title.str.contains('coronary artery', case=False)].icd_code.to_list()
chf_icd = data_d_icd_diagnoses[(data_d_icd_diagnoses.long_title.str.contains('heart failure', case=False))\
                                &(data_d_icd_diagnoses.long_title.str.contains('congestive', case=False))].icd_code.to_list()
liver_icd = data_d_icd_diagnoses[(data_d_icd_diagnoses.long_title.str.contains('liver', case=False))\
                                &(data_d_icd_diagnoses.long_title.str.contains('disease', case=False))].icd_code.to_list()
mi_icd = data_d_icd_diagnoses[(data_d_icd_diagnoses.long_title.str.contains('myocardial', case=False))\
                                &(data_d_icd_diagnoses.long_title.str.contains('infarction', case=False))\
                                    &~(data_d_icd_diagnoses.long_title.str.contains('not resulting', case=False))].icd_code.to_list()
diabetes_icd = data_d_icd_diagnoses[data_d_icd_diagnoses.long_title.str.contains('diabetes', case=False)].icd_code.to_list()
hypertension_icd = data_d_icd_diagnoses[data_d_icd_diagnoses.long_title.str.contains('hypertension', case=False)].icd_code.to_list()
pv_icd = data_d_icd_diagnoses[(data_d_icd_diagnoses.long_title.str.contains('peripheral', case=False))\
                                &(data_d_icd_diagnoses.long_title.str.contains('vascular', case=False))].icd_code.to_list()

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    #return lst3
    return lst3

In [ ]:
admissions = data_icustays.hadm_id.unique()
D = data_diagnoses.merge(data_d_icd_diagnoses).merge(data_admissions[['subject_id', 'hadm_id', 'admittime']])
diag_l = {'cirrhosis':cirrhosis_icd, 'cad':cad_icd, 'chf':chf_icd, 'liver':liver_icd, 'mi':mi_icd, \
                'diabetes':diabetes_icd, 'hypertension':hypertension_icd, 'pv':pv_icd}

In [ ]:
%%time
prev_diags_dict_l = []
i = 0

for adm in admissions:
    p = data_admissions[data_admissions.hadm_id==adm].subject_id.values[0]
    admittime = data_admissions[data_admissions.hadm_id==adm].admittime.values[0]
    if i % 1000 == 0:
        print(f'{i}/{len(admissions)}')
    prev_diags = list(D[(D.subject_id==p) & (D.admittime < admittime)].icd_code.values)
    prev_diags_dict = {'hadm_id':adm}
    for name, d in diag_l.items():
        if intersection(prev_diags, d):
            prev_diags_dict[name] = 1
        else:
            prev_diags_dict[name] = 0
    prev_diags_dict_l.append(prev_diags_dict)
    i += 1

In [ ]:
len(prev_diags_dict_l)

In [ ]:
data_diags_xgb = pd.DataFrame(prev_diags_dict_l)
data_diags_xgb.sum().sort_values()

In [ ]:
data_diags_xgb.head()

Save

In [ ]:
with open(XGB_DATA_PATH + 'data_diags_xgb.pkl', 'wb') as f:
    pickle.dump(data_diags_xgb, f, protocol=pickle.HIGHEST_PROTOCOL)

## Medications

In [ ]:
data_inputevents.head(2)

In [ ]:
with open(PKL_PATH + 'pid_medications_icu.pkl', 'rb') as f:
    pid_medications = pickle.load(f)

# pid_medications = pid_medications.merge(data_icustays[['subject_id', 'hadm_id', 'stay_id', 'intime']])
pid_medications.head()

In [ ]:
# get names for NSAIDs medications M01
M01_names = list(ndc_map[ndc_map.atc4.str.contains('M01', case=False)]\
                .drop_duplicates(subset='in_name', keep='first').in_name.values)

print('M01_names: ', len(M01_names))

# get names for lipid-lowering  medications C10
C10_names = list(ndc_map[ndc_map.atc4.str.contains('C10', case=False)]\
                .drop_duplicates(subset='in_name', keep='first').in_name.values)
print('C10_names', len(C10_names))

# get names for antithrombotic medications
B01_names = list(ndc_map[ndc_map.atc4.str.contains('B01', case=False)]\
                .drop_duplicates(subset='in_name', keep='first').in_name.values)

print('B01_names', len(B01_names))

# get names for diabetes medications
diabetes_A10_names = list(ndc_map[ndc_map.atc4.str.contains('A10', case=False)]\
                .drop_duplicates(subset='in_name', keep='first').in_name.values)

print('A10_names', len(diabetes_A10_names))

# get names for diuretics medications
C03_names = list(ndc_map[ndc_map.atc4.str.contains('C03', case=False)]\
                .drop_duplicates(subset='in_name', keep='first').in_name.values)
print('C03_names', len(C03_names))

# get names for antihypertensives medications
C02_names = list(ndc_map[ndc_map.atc4.str.contains('C02', case=False)]\
                .drop_duplicates(subset='in_name', keep='first').in_name.values)
print('C02_names', len(C02_names))

# get names for V08 contrast media medications
V08_names = list(ndc_map[ndc_map.atc4.str.contains('V08', case=False)]\
                .drop_duplicates(subset='in_name', keep='first').in_name.values)
print('V08_names', len(V08_names))

# get names for lipid-lowering  medications C09 agent acting on the renin-antiotensin system
C09_names = list(ndc_map[ndc_map.atc4.str.contains('C09', case=False)]\
                .drop_duplicates(subset='in_name', keep='first').in_name.values)
print('C09_names', len(C09_names))

In [ ]:
stays = data_icustays[data_icustays.los>2].stay_id

In [ ]:
# looping over stays to get medications for a specific day in ICU (day 0)
medication_mappings = [M01_names, C10_names, B01_names, diabetes_A10_names, C03_names, C02_names, C09_names]
stay_medication_mapping_dict = {}
# day in ICU to get medication info for
icu_day = 0 
i = 0

for stay in stays:
    if i % 1000==0:
        print(f'{i}/{len(stays)}')
  
    stay_meds = pid_medications[(pid_medications.stay_id==stay)&(pid_medications.icu_day_id==icu_day)]
    med_list = []
    for medication_mapping, medication_code in zip(medication_mappings, ['M01', 'C10', 'B01', 'A10', 'C03', 'C02', 'C09']):
        # print(medication_code)
        medication_on_day = stay_meds[(stay_meds.medications_names.str.contains('|'.join(medication_mapping), case=False, na=''))]
        if len(medication_on_day) > 0:
            med_list.append(1)
        else:
            med_list.append(0)
    stay_medication_mapping_dict[stay] = med_list 
    i += 1

In [ ]:
data_medications_xgb = pd.DataFrame.from_dict(stay_medication_mapping_dict, orient='index')
data_medications_xgb.columns = ['M01', 'C10', 'B01', 'A10', 'C03', 'C02', 'C09']
data_medications_xgb = data_medications_xgb.drop(columns=['C10', 'A10', 'C09'])
data_medications_xgb = data_medications_xgb.reset_index().rename(columns={'index':'stay_id'})

In [ ]:
data_medications_xgb.head()

In [ ]:
data_medications_xgb.sum()

 Check:

In [ ]:
stay_meds = pid_medications[(pid_medications.stay_id==33685454)&(pid_medications.icu_day_id==icu_day)]
stay_meds[(stay_meds.medications_names.str.contains('|'.join(C03_names), case=False, na=''))].medications_names.values

In [ ]:
C03_names

Save

In [ ]:
with open(XGB_DATA_PATH + 'data_medications_xgb.pkl', 'wb') as f:
    pickle.dump(data_medications_xgb, f, protocol=pickle.HIGHEST_PROTOCOL)

## Compiling XGB dataset

### Load dataframes

In [ ]:
XGB_DATA_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/xgb/dataframes/'

In [ ]:
with open(PKL_PATH + 'aki_status_icu.pkl', 'rb') as f:
    aki_status_icu = pickle.load(f)
aki_status_icu = aki_status_icu.groupby(['subject_id',	'hadm_id',	'stay_id', 'icu_day_id', 'specimen_id']).sum().reset_index()
aki_status_icu['AKI_1'] = (aki_status_icu['AKI_1_scr'] + aki_status_icu['AKI_1_urine']).astype(bool).astype(int)
aki_status_icu['AKI_2'] = (aki_status_icu['AKI_2_scr'] + aki_status_icu['AKI_2_urine']).astype(bool).astype(int)
aki_status_icu['AKI_3'] = (aki_status_icu['AKI_3_scr'] + aki_status_icu['AKI_3_urine']).astype(bool).astype(int)
aki_status_icu_day1 = aki_status_icu[aki_status_icu.icu_day_id==1]
aki_status_icu_day1.head(5)

In [ ]:
with open(XGB_DATA_PATH + 'data_demographics_omr.pkl', 'rb') as f:
    data_demographics_omr = pickle.load(f)
data_demographics_omr.head()

In [ ]:
p = 15926317
data_demographics[data_demographics.subject_id==p]

In [ ]:
with open(XGB_DATA_PATH + 'data_labs_xgb.pkl', 'rb') as f:
    data_labs_xgb = pickle.load(f)
data_labs_xgb['subject_id'] = data_labs_xgb['subject_id'].astype('int64')
data_labs_xgb['stay_id'] = data_labs_xgb['stay_id'].astype('int64')
data_labs_xgb['hadm_id'] = data_labs_xgb['hadm_id'].astype('int64')
# take only day 0 of ICU stay
data_labs_xgb = data_labs_xgb[data_labs_xgb.icu_day_id==0]
print(data_labs_xgb.shape)
data_labs_xgb.head()

In [ ]:
with open(XGB_DATA_PATH + 'data_vitals_xgb.pkl', 'rb') as f:
    data_vitals_xgb = pickle.load(f)
data_vitals_xgb['subject_id'] = data_vitals_xgb['subject_id'].astype('int64')
data_vitals_xgb['stay_id'] = data_vitals_xgb['stay_id'].astype('int64')
data_vitals_xgb['hadm_id'] = data_vitals_xgb['hadm_id'].astype('int64')
# take only day 0 of ICU stay
data_vitals_xgb = data_vitals_xgb[data_vitals_xgb.icu_day_id==0]
print(data_vitals_xgb.shape)
data_vitals_xgb.head()

In [ ]:
with open(XGB_DATA_PATH + 'data_output_xgb.pkl', 'rb') as f:
    data_output_xgb = pickle.load(f)
data_output_xgb = data_output_xgb[data_output_xgb.icu_day_id==0]
print(data_output_xgb.shape)
data_output_xgb.head()

In [ ]:
with open(XGB_DATA_PATH + 'data_diags_xgb.pkl', 'rb') as f:
    data_diags_xgb = pickle.load(f)
data_diags_xgb.head()

In [ ]:
with open(XGB_DATA_PATH + 'data_medications_xgb.pkl', 'rb') as f:
    data_medications_xgb = pickle.load(f)
data_medications_xgb.head()

In [ ]:
with open(PKL_PATH + 'pid_demographics_icu.pkl', 'rb') as f:
    pid_demographics_icu =  pickle.load(f)
print('unique stays: ', pid_demographics_icu.stay_id.unique().shape[0])
pid_demographics_icu.head()

### Remove columns with a lot of nan values:

Labs

In [ ]:
# print how many NaN values in each column
for col, num_nans in data_labs_xgb.isna().sum().sort_values().iteritems():
    print(col, num_nans)

In [ ]:
data_labs_xgb = data_labs_xgb[data_labs_xgb.isna().sum().sort_values().index[:40]]

In [ ]:
data_labs_xgb.head()

Vitals

In [ ]:
# print how many NaN values in each column
for col, num_nans in data_vitals_xgb.isna().sum().sort_values().iteritems():
    print(col, num_nans)

In [ ]:
data_vitals_xgb = data_vitals_xgb[data_vitals_xgb.isna().sum().sort_values().index[:18]]

In [ ]:
data_vitals_xgb.head()

### Merge dataframes:

In [ ]:
data_xgb = data_demographics_omr.merge(data_labs_xgb[data_labs_xgb.icu_day_id==0], on=['subject_id', 'hadm_id', 'stay_id'], how='outer')\
                        .merge(data_vitals_xgb[data_vitals_xgb.icu_day_id==0], on=['subject_id', 'hadm_id', 'stay_id'], how='outer')\
                            .merge(data_output_xgb[data_output_xgb.icu_day_id==0], on=['subject_id', 'hadm_id', 'stay_id'], how='outer')

We merge data with demographics from pid_demographics_icu dataset:

In [ ]:
pid_demographics_icu.head(1)

In [ ]:
data_xgb = data_xgb.merge(pid_demographics_icu[['subject_id',	'hadm_id', 'race', 'gender', 'age']], on=['subject_id', 'hadm_id'])

In [ ]:
print(data_xgb.icu_day_id.unique())
print(data_xgb.icu_day_id_x.unique())
print(data_xgb.icu_day_id_y.unique())

In [ ]:
data_xgb['icu_day_id'] = data_xgb['icu_day_id'].fillna(0)
data_xgb = data_xgb.drop(['icu_day_id', 'icu_day_id_x', 'icu_day_id_y'], axis=1)

In [ ]:
p = 15926317
data_xgb[data_xgb.subject_id==p]

In [ ]:
data_omr[data_omr.subject_id==p]

In [ ]:
data_icustays[data_icustays.subject_id==p]

In [ ]:
for k,v in data_xgb.isna().sum().sort_values().iteritems():
    print(k, v)

We merge data for the 0 day in ICU and labels for the 1st day in ICU:

In [ ]:
print(data_xgb.shape)
data_xgb.head(1)

In [ ]:
print(aki_status_icu_day1.shape)
aki_status_icu_day1.head(1)

In [ ]:
data_xgb = data_xgb.merge(aki_status_icu_day1[['subject_id',	'hadm_id',	'stay_id', 'AKI_1',	'AKI_2', 'AKI_3']])
print(data_xgb.shape)
data_xgb.head()

In [ ]:
data_xgb = data_xgb.drop_duplicates(subset=['subject_id',	'hadm_id', 'stay_id'])

In [ ]:
print(f'unique patients: {data_xgb.subject_id.unique().shape[0]}')
print(f'unique admissions: {data_xgb.hadm_id.unique().shape[0]}')
print(f'unique ICU stays: {data_xgb.stay_id.unique().shape[0]}')

In [ ]:
data_xgb

In [ ]:
pid_demographics_icu[pid_demographics_icu.subject_id==19926342]

Save

In [ ]:
XGB_DATA_PATH

In [ ]:
with open(XGB_DATA_PATH + 'data_xgb.pkl', 'wb') as f:
    pickle.dump(data_xgb, f, protocol=pickle.HIGHEST_PROTOCOL)

# Explore LIME explanations

In [ ]:
with open(PKL_PATH + 'chartevents_labs.pkl', 'rb') as f:
    chartevents_labs = pickle.load(f)

In [ ]:
with open(PKL_PATH + 'chartevents_vitals.pkl', 'rb') as f:
    chartevents_vitals = pickle.load(f)

In [ ]:
with open(PKL_PATH+'data_medications_icu.pkl', 'rb') as f:
    data_medications = pickle.load(f)

In [ ]:
chartevents_labs[chartevents_labs.itemid.isin([220228, 225625])].drop_duplicates('itemid')

In [ ]:
chartevents_labs[chartevents_labs.itemid==220228].head(1)

In [ ]:
chartevents_vitals[chartevents_vitals.isin([220045])].drop_duplicates('itemid')

In [ ]:
data_d_icd_diagnoses[data_d_icd_diagnoses.icd_code.isin(['5920', '25053', '2724', 'E8798', '2572', 'Z87891', 'E0500', '7908', 'I2720', 'Z8505'])]

In [ ]:
data_medications[data_medications.itemid.isin([223258, 227522, 225166, 223262])].drop_duplicates('itemid')